In [1]:
import os
import pandas as pd
import math
import numpy as np
import gc
from datetime import datetime

In [2]:
################################################################################
# Data
################################################################################

# DATA_DIR =  '../data/'
DATA_DIR = r'C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data'

#project variables
# EFFECTIVE_DATA_FILE_PATH = os.path.join(DATA_DIR, 'TMS_EFFECTIVES_2023_04_05_06_V240206.csv')
# JUSTIF_DATA_FILE_PATH = os.path.join(DATA_DIR, 'TMS_JUSTIF_2023_04_05_06_V240430.csv')
EFFECTIVE_DATA_FILE_PATH = os.path.join(DATA_DIR, 'TMS_EFFECTIVES_2023_04_05_06_V240206.csv')
JUSTIF_DATA_FILE_PATH = os.path.join(DATA_DIR, 'JUSTIF_test.csv')
MINUTE= 5
#T_DATATYPE = 'int64'
T_DATATYPE ='int32'

TIME_FREQUENCY='5min'
#TIME_FORMAT='%Y-%m-%d %H:%M'
TIME_FORMAT='%H:%M'

EFFECTIVE_DF_CLEANED_FILE_PATH = os.path.join(DATA_DIR, 'Effective_df_cleaned_' + TIME_FREQUENCY + '.parquet')
EFFECTIVE_DF_DEPARTURE_FILE_PATH = os.path.join(DATA_DIR, 'Effective_df_D_i_' + TIME_FREQUENCY + '.parquet')

EFFECTIVE_DF_DA_IT_SMALL_FILE_PATH = os.path.join(DATA_DIR, 'Effective_df_DA_it_small' + TIME_FREQUENCY + '.parquet')

EFFECTIVE_DF_DA_IT_W_FREIGHT_FILE_PATH  = os.path.join(DATA_DIR, 'Effective_df_DA_it_with_freight_' + TIME_FREQUENCY + '.parquet')
EFFECTIVE_DF_A_IT_FILE_PATH = os.path.join(DATA_DIR, 'Effective_df_a_it_' + TIME_FREQUENCY + '.parquet')
EFFECTIVE_DF_A_IT_W0_FILE_PATH = os.path.join(DATA_DIR, 'Effective_df_a_it_w0_' + TIME_FREQUENCY + '.parquet')
EFFECTIVE_DF_D_IT_FILE_PATH = os.path.join(DATA_DIR, 'Effective_df_d_it_' + TIME_FREQUENCY + '.parquet')

EFFECTIVE_DF_STATS_FILE_PATH = os.path.join(DATA_DIR,'effective_df_statistics_'+ TIME_FREQUENCY + '.csv')
EFFECTIVE_DF_STATS_DA_IT_FILE_PATH = os.path.join(DATA_DIR,'effective_df_D_it_with_freight_statistics_'+ TIME_FREQUENCY + '.csv')

FUNCTION_S_R_W0_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_function_S_r_w0_' + TIME_FREQUENCY + '.parquet')

## Delay file path
TOTAL_DELAY_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_total_delay_' + TIME_FREQUENCY + '.parquet')
PRIMARY_DELAY_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_primary_delay_' + TIME_FREQUENCY + '.parquet')
PRIMARY_DELAY_CLEANED_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_primary_delay_cleaned_' + TIME_FREQUENCY + '.parquet')
SECONDARY_DELAY_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_secondary_delay_' + TIME_FREQUENCY + '.parquet')
SECONDARY_DELAY_CLEANED_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_secondary_delay_cleaned_' + TIME_FREQUENCY + '.parquet')

ENDODATA_WITH_G_FILE_PATH  = os.path.join(DATA_DIR, 'Endogenous_data_with_valid_G_values_' + TIME_FREQUENCY + '.parquet')
ENDODATA_WITH_G_PRI_SEC_DELAY_FILE_PATH  = os.path.join(DATA_DIR,'Endogenous_data_with_valid_G_values_prim_sec_delay_' + TIME_FREQUENCY + '.parquet')
REGRESSION_INPUT_VALID_G_FILE_PATH =  os.path.join(DATA_DIR, 'Endogenous_data_with_valid_G_values_prim_sec_delay_observed_rec_' + TIME_FREQUENCY + '.csv')
REGRESSION_INPUT_VALID_G_DALTA_FILE_PATH =  os.path.join(DATA_DIR, 'Endogenous_data_with_valid_G_values_prim_sec_delay_observed_Delta_rec_' + TIME_FREQUENCY + '.csv')

## The below can be deleted
REGRESSION_INPUT_ALL_ACTIVE_T_W_DUMMIES_FILE_PATH = os.path.join(DATA_DIR, 'Data_for_regression_input_all_active_t_w_dummies_' + TIME_FREQUENCY + '.csv')


In [3]:
################################################################################
# This is a helper function to read parquet file. It is not used for the moment, 
# but it can be used in case we need to read the parquet files.
################################################################################

def read_function_data(fileName):
    temp_df = pd.read_parquet(fileName, engine='fastparquet')
    #print(temp_df.dtypes)
    return temp_df

# Explore and check the correctness of effective data

In [4]:
effective_df_org = pd.read_csv(EFFECTIVE_DATA_FILE_PATH)

print(effective_df_org.shape)
print(effective_df_org.columns)
effective_df_org.head(2)

## The below variable of EFFECTIVE_DF_COLS_TO_DROP is used in the function 'preprocess_effective' to drop unused cols
EFFECTIVE_DF_COLS_TO_DROP = ['hd_messageidentifier', 'hd_timestamp', 'tr_bk',
       'tr_bk_domesticdeparturedate', 'tr_responsible', 'tr_priority',
       'tr_is_initialtimetabletype', 'tr_ismodifiedinrealtime', 'tr_isundo',
       'info_maxspeed', 'point_trackingtype',
       'planned_reservetime_or_buffertim',
       'point_ptcar_symbolicname', 'point_ptcar_shortnamenl',
       'point_ptcar_gpslatitude', 'point_ptcar_gpslongitude', 'point_ismanual',
       'point_macro_time', 'point_micro_toptrefsymbolicname']


## The below variable EFFECTIVE_DF_COLS_NEEDED is just to keep track of all the needed variables 
#for sanity check
## when we get new dataset
EFFECTIVE_DF_COLS_NEEDED= ['tr_bk_number', 'point_ptcarid','point_macro_time','point_macro_delay',
                           'point_movement', 'info_commercialtype', 'point_ptcarsequencenumber', 
                           'tr_isempty', 'tr_traingroup', 'point_micro_time', 'point_micro_fromptrefsymbolicnam']


(22367077, 32)
Index(['hd_messageidentifier', 'hd_timestamp', 'tr_bk',
       'tr_bk_domesticdeparturedate', 'tr_bk_number', 'tr_isempty',
       'tr_traingroup', 'tr_responsible', 'tr_priority',
       'tr_is_initialtimetabletype', 'tr_ismodifiedinrealtime', 'tr_isundo',
       'info_commercialtype', 'info_maxspeed', 'point_trackingtype',
       'point_movement', 'point_ptcarsequencenumber', 'point_isunexpected',
       'planned_status', 'planned_timetabletype',
       'planned_reservetime_or_buffertim', 'point_ptcarid',
       'point_ptcar_symbolicname', 'point_ptcar_shortnamenl',
       'point_ptcar_gpslatitude', 'point_ptcar_gpslongitude', 'point_ismanual',
       'point_macro_time', 'point_macro_delay', 'point_micro_time',
       'point_micro_fromptrefsymbolicnam', 'point_micro_toptrefsymbolicname'],
      dtype='object')


### clean/preprocess effective data

In [5]:
################################################################################
# The following function prints the number of NA per columns in the EFFECTIVE data set
################################################################################

def check_na_cols(temp_df):
    
    """
    Function to verify the cleanliness of the effective dataframe
    by checking for NaN values in specific columns.
    """
    # Count NaN values in each column
    tr_bk_number_nan_count = temp_df['tr_bk_number'].isna().sum()
    tr_bk_nan_count = temp_df['tr_bk'].isna().sum()
    tr_type_nan_count = temp_df['info_commercialtype'].isna().sum()
    micro_nan_count = temp_df['point_micro_time'].isna().sum()
    ptcar_nan_count = temp_df['point_ptcarid'].isna().sum()
    macro_nan_count = temp_df['point_macro_time'].isna().sum()
    macro_delay_nan_count = temp_df['point_macro_delay'].isna().sum()
    point_movement_nan_count = temp_df['point_movement'].isna().sum()
    
    # Print the results
    print("Total rows in effective data:", temp_df.shape[0])
    print("Train number NaN count:", tr_bk_number_nan_count)
    print("Train ID NaN count:", tr_bk_nan_count)
    print("Train type NaN count:", tr_type_nan_count)
    print("Micro time NaN count:", micro_nan_count)
    print("PT car NaN count:", ptcar_nan_count)
    print("Macro time NaN count:", macro_nan_count)
    print("Macro delay NaN count:", macro_delay_nan_count)
    print("Point Movement NaN count:", point_movement_nan_count)
    
check_na_cols(effective_df_org) 

Total rows in effective data: 22367077
Train number NaN count: 0
Train ID NaN count: 0
Train type NaN count: 501530
Micro time NaN count: 4249963
PT car NaN count: 0
Macro time NaN count: 0
Macro delay NaN count: 103890
Point Movement NaN count: 0


In [6]:
################################################################################
# Function to convert date format of EFFECTIVE data to match justif data, 
# the new timestamp column created is named as 'formatted_point_macro_time',
# we use 'formatted_point_macro_time' to create 't',r0,r1,r2, 

# we convert the datatype for tr_bk_number (which is the train identifier) and point_ptcarid (which is the station identifier) 
# from double to int. Next, we fill the nan values with 0 for point_macro_delay
# We rename the column names for 
    # tr_bk_number as 'i'
    # point_ptcarid as 's'
    # point_movement as 'm_org'
    # point_macro_delay as'w(i,t)'
################################################################################

from datetime import datetime
import pytz

def preprocess_effective(temp_df):

    # Function to preprocess and clean the effective data 
    # It returns a reduced data frame with 6 columns

    temp_df.dropna(how='all', inplace=True)
    print(temp_df.shape)
    
    # Convert 'point_macro_time' to datetime format and sort
    temp_df['point_macro_time'] = pd.to_datetime(temp_df['point_macro_time'], format='%d%b%Y:%H:%M:%S.%f')
    temp_df = temp_df.sort_values(by='point_macro_time').reset_index(drop=True)
    temp_df['day_name'] = temp_df['point_macro_time'].dt.day_name()
    
    # conver the point_macro_time to the date format that matches justif data
    temp_df['formatted_point_macro_time'] = pd.to_datetime(temp_df['point_macro_time']).dt.tz_localize(pytz.timezone('Europe/Paris'))
    
    # Convert 'tr_bk_number' and 'point_ptcarid' to integer format
    temp_df['tr_bk_number'] = temp_df['tr_bk_number'].astype(int)
    temp_df['info_commercialtype'] = temp_df['info_commercialtype'].fillna('Ntype')
    temp_df['point_ptcarid'] = temp_df['point_ptcarid'].astype(int)
    temp_df['point_macro_delay'] = temp_df['point_macro_delay'].fillna(0)
    temp_df['point_ptcarsequencenumber'] = pd.to_numeric(temp_df['point_ptcarsequencenumber'], errors='coerce').astype('Int64')
    
    # Dictionary of old and new column names
    new_column_names = {
        'tr_bk_number': 'i',
        'info_commercialtype': 'tr_type',
        'point_ptcarid': 's',
        'point_movement': 'm_org',
        'point_macro_delay': 'w(i,t)'
    }
        
    
    # Rename columns
    temp_df.rename(columns=new_column_names, inplace=True)
    
    # drop unused columns
    temp_df.drop(EFFECTIVE_DF_COLS_TO_DROP, axis=1, inplace=True)
    
    return temp_df

# Note that 'effective_df' is a reduced data frame with 9 columns
effective_df = preprocess_effective(effective_df_org)
print(effective_df.shape )
effective_df.head(4)

(22367077, 32)
(22367077, 15)


,i,tr_isempty,tr_traingroup,tr_type,m_org,point_ptcarsequencenumber,point_isunexpected,planned_status,planned_timetabletype,s,"w(i,t)",point_micro_time,point_micro_fromptrefsymbolicnam,day_name,formatted_point_macro_time
0,-17831,0,FREIGHT,HKM5,out,109,0,ACTIVE,IMPORTANT_PASSAGE,1084,-5317.00,01APR2023:00:01:27.000000,276011.0,Friday,2023-03-31 23:59:23+02:00
1,-86432,0,FREIGHT,HKM5,out,103,0,ACTIVE,IMPORTANT_PASSAGE,1195,-3695.00,01APR2023:00:00:33.000000,402071.0,Friday,2023-03-31 23:59:25+02:00
2,-8436,0,FREIGHT,Ntype,out,101,0,ACTIVE,COMMERCIAL_STOP,456,-112.36,01APR2023:00:00:11.000000,592792.0,Saturday,2023-04-01 00:00:07+02:00
3,-85230,0,PASSENGER,L,out,3,0,ACTIVE,COMMERCIAL_STOP,349,130.00,NaN,NaN,Saturday,2023-04-01 00:00:10+02:00


In [7]:
################################################################################
# we are deleting the effective_df_org to save memory
################################################################################

del effective_df_org
gc.collect()

145

In [8]:
################################################################################
## The effective_df contains 2 records from march 31st in the point_micro_time, 
# As our dataset is already sorted, we are dropping those first 2 rows here and reseting the index
################################################################################

effective_df = effective_df.iloc[2:].reset_index(drop=True)

print(effective_df.shape)
effective_df.head(2)

(22367075, 15)


,i,tr_isempty,tr_traingroup,tr_type,m_org,point_ptcarsequencenumber,point_isunexpected,planned_status,planned_timetabletype,s,"w(i,t)",point_micro_time,point_micro_fromptrefsymbolicnam,day_name,formatted_point_macro_time
0,-8436,0,FREIGHT,Ntype,out,101,0,ACTIVE,COMMERCIAL_STOP,456,-112.36,01APR2023:00:00:11.000000,592792.0,Saturday,2023-04-01 00:00:07+02:00
1,-85230,0,PASSENGER,L,out,3,0,ACTIVE,COMMERCIAL_STOP,349,130.00,NaN,NaN,Saturday,2023-04-01 00:00:10+02:00


In [9]:
effective_df.head(20)

,i,tr_isempty,tr_traingroup,tr_type,m_org,point_ptcarsequencenumber,point_isunexpected,planned_status,planned_timetabletype,s,"w(i,t)",point_micro_time,point_micro_fromptrefsymbolicnam,day_name,formatted_point_macro_time
0,-8436,0,FREIGHT,Ntype,out,101,0,ACTIVE,COMMERCIAL_STOP,456,-112.36,01APR2023:00:00:11.000000,592792.0,Saturday,2023-04-01 00:00:07+02:00
1,-85230,0,PASSENGER,L,out,3,0,ACTIVE,COMMERCIAL_STOP,349,130.00,NaN,NaN,Saturday,2023-04-01 00:00:10+02:00
2,-87651,1,PASSENGER,Ntype,out,101,0,ACTIVE,OPERATIONAL_STOP,78,-214.68,01APR2023:00:00:41.000000,780041.0,Saturday,2023-04-01 00:00:25+02:00
3,-91842,1,PASSENGER,HPV,in,103,0,ACTIVE,IMPORTANT_PASSAGE,871,-1288.00,01APR2023:00:00:23.000000,273281.0,Saturday,2023-04-01 00:00:32+02:00
4,-91842,1,PASSENGER,HPV,out,103,0,ACTIVE,IMPORTANT_PASSAGE,871,-1288.00,NaN,NaN,Saturday,2023-04-01 00:00:32+02:00
5,-17831,0,FREIGHT,HKM5,in,110,0,ACTIVE,IMPORTANT_PASSAGE,1622,-5300.04,01APR2023:00:01:27.000000,276011.0,Saturday,2023-04-01 00:00:39+02:00
6,-17831,0,FREIGHT,HKM5,out,110,0,ACTIVE,IMPORTANT_PASSAGE,1622,-5300.04,01APR2023:00:01:27.000000,276011.0,Saturday,2023-04-01 00:00:39+02:00
7,-91842,1,PASSENGER,HPV,out,104,0,ACTIVE,IMPORTANT_PASSAGE,883,-1264.00,NaN,NaN,Saturday,2023-04-01 00:00:56+02:00
8,-91842,1,PASSENGER,HPV,in,104,0,ACTIVE,IMPORTANT_PASSAGE,883,-1264.00,NaN,NaN,Saturday,2023-04-01 00:00:56+02:00
9,-85230,0,PASSENGER,L,out,3,0,ACTIVE,COMMERCIAL_STOP,349,187.00,NaN,NaN,Saturday,2023-04-01 00:01:07+02:00


In [10]:
################################################################################
# Function to Extract strting time of our data from the column formatted_point_macro_time in effective data
#  The function Sets the global variable 'start_timestamp' to the starting timestamp extracted from the DataFrame
################################################################################

global start_timestamp 

def extract_start_timestamp_from_effective(df,time_column):
    
    """
    Extracts the starting timestamp from the specified column in the DataFrame.
    The starting timestamp is set as a global variable 'start_timestamp' for
    accessibility from anywhere after calling this function..

    Args:
    - df: pandas DataFrame containing the timestamps.
    - time_column: str, the name of the column containing timestamps that is same as justif timestamp format.

    Returns:
    - pd.Timestamp: The starting timestamp extracted from the DataFrame.
    """

    start_timestamp = df[time_column].iloc[0].normalize()
    start_timestamp = start_timestamp+ pd.Timedelta(hours=0)
    return start_timestamp
     

start_timestamp = extract_start_timestamp_from_effective(effective_df,'formatted_point_macro_time')
start_timestamp

Timestamp('2023-04-01 00:00:00+0200', tz='Europe/Paris')

## Prepare timestamp column for delay in the Effective data set

We define $r_0(t)$, $r_1(t)$, and $r_2(t)$ as the respective time of the day (in minutes), the day of the week (from one to seven) and month of the year (from one to seven), corresponding to data time $t$. We introduce the notation $\boldsymbol{r}(t) = (r_0(t), r_1(t), r_2(t))$.

The following code is to create $r_0(t)$ in blocks of 5 minutes.

In [11]:
################################################################################
# Function to create r0, r1 and r2 by removing seconds and timezone from the  
# columns 'formatted_point_macro_time' in the effective_df
#
################################################################################

def Append_r0_r1_r2(df, time_column):
    
    """
    This function adds a new column to df by converting timestamps in a specified column to simplified time units where each 5-minute interval is considered a unit.
    
    Input Args:
    - df: pandas DataFrame containing the timestamps.
    - time_column: the name of the column containing timestamps in ISO 8601 format.
   
    Output Returns:
    - A pandas DataFrame contructed from df, with an additional column 'simplified_t' containing the simplified time units.
    - r0: the name of the 'minute' column that this function is going to append to df
    - r1 the name of the 'week' column that this function is going to append to df
    - r2 the name of the 'month' column that this function is going to append to df

    """

    # Convert the time_column to datetime if not done earlier
    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        df[time_column] = pd.to_datetime(df[time_column])

    # Extract hours and minutes
    hours = df[time_column].dt.hour
    minutes = df[time_column].dt.minute

    # Calculate total minutes from the start of the day
    total_minutes = hours * 60 + minutes

    # Convert to simplified time units
    r0 = np.ceil(total_minutes // MINUTE).astype(int)

    r1 = df[time_column].dt.dayofweek + 1  # Adding 1 to shift from 0-6 to 1-7
    r2 = df[time_column].dt.month

    return r0.astype('int16'),r1.astype('int8'),r2.astype('int8')


# After this operation 'effectiv_df' will have 12 columns (the 9 original columns + r0 + r1 + r2)

r0, r1, r2 = Append_r0_r1_r2(effective_df, 'formatted_point_macro_time')
effective_df['r0'] = r0
effective_df['r1'] = r1
effective_df['r2'] = r2
effective_df.head(2)

,i,tr_isempty,tr_traingroup,tr_type,m_org,point_ptcarsequencenumber,point_isunexpected,planned_status,planned_timetabletype,s,"w(i,t)",point_micro_time,point_micro_fromptrefsymbolicnam,day_name,formatted_point_macro_time,r0,r1,r2
0,-8436,0,FREIGHT,Ntype,out,101,0,ACTIVE,COMMERCIAL_STOP,456,-112.36,01APR2023:00:00:11.000000,592792.0,Saturday,2023-04-01 00:00:07+02:00,0,6,4
1,-85230,0,PASSENGER,L,out,3,0,ACTIVE,COMMERCIAL_STOP,349,130.00,NaN,NaN,Saturday,2023-04-01 00:00:10+02:00,0,6,4


In [12]:
effective_df['point_micro_time'] = (
    pd.to_datetime(effective_df['point_micro_time'], format="%d%b%Y:%H:%M:%S.%f")
      .dt.tz_localize("Europe/Paris")
)

In [13]:
################################################################################
# This function takes timestamp column as input  and returns 't' (in blocks of 5 minutes) 
# by removing seconds and timezone. For ex. we can pass the columns 'formatted_point_macro_time' 
# in the effecctive_df data, as input to the function.
#
################################################################################

def Append_t(df_time_column):
    """
    Converts the input timestamps to simplified time units where each 5-minute interval is considered a unit.

    Args:
    - df_time_column: pandas Series, the column containing timestamps in ISO 8601 format.

    Returns:
    -pandas Series: A Series containing the converted values of the timestamp column.

    """

    # Calculate the difference in minutes from the starting timestamp
    diff_in_minutes_from_start_timestamp= ((df_time_column - start_timestamp).dt.total_seconds() / 60).round(2)

    # Divide the difference by 5 to get the running number in minutes
    current_minute= (diff_in_minutes_from_start_timestamp// MINUTE).astype(T_DATATYPE) 

    return current_minute

# After this operation 'effectiv_df' will have 13 columns (the 12 original columns + t)
effective_df['t'] = Append_t(effective_df['formatted_point_macro_time'])

In [14]:
################################################################################
# The following functions `get_r0(t)`, `get_r1(t)`, and `get_r2(t)` are helper functions that take 
# an integer 't' or a panda series(column) containing simplified time units 't' (in blocks of 5 minutes) as input 
# and returns an integer or a panda series containing
# - r0: The minute unit (0 to 287, where 0 represents the first minute and 287 represents the last minute of a day).
# - r1: The day of the week (1 to 7, where 1 is Monday and 7 is Sunday).
# - r2: The month (1 to 12, where 1 is January and 12 is December).
################################################################################

def get_r0(t):
    """
    Takes an integer t (simplified time units) or pd.Series 't'.

    Args:
    - t: int or pd.series format the simplified time unit.

    Returns:
    - r0: returns int if 't' is int or pd.series if 't' is pd.series
    """
    if type(t) == int:

        timestamp = start_timestamp + pd.Timedelta(minutes=t * MINUTE)
        total_minutes = (timestamp.hour * 60) + timestamp.minute

        # Convert to simplified time units
        r0 = np.ceil(total_minutes // MINUTE).astype(int)

    else:

        timedelta_values = pd.to_timedelta(t * MINUTE, unit='m')
        timestamp_val = start_timestamp + timedelta_values
    
        r0 = np.ceil((timestamp_val.dt.hour * 60 + timestamp_val.dt.minute) // MINUTE).astype(int)
    
    return r0

#---------------------------------

def get_r1(t):   
    """
    Takes an integer t (simplified time units) or pd.Series 't'.

    Args:
    - t: int or pd.series format the simplified time unit.

    Returns:
    - r1: returns int if 't' is int or pd.series if 't' is pd.series
    """
    if type(t) == int:
        
        timestamp = start_timestamp + pd.Timedelta(minutes=t * MINUTE)
        r1 = timestamp.dayofweek + 1

    else:
    
        timedelta_values = pd.to_timedelta(t * MINUTE, unit='m')
        timestamp_val = start_timestamp + timedelta_values
        r1 = timestamp_val.dt.dayofweek + 1

    return r1

#---------------------------------

    
def get_r2(t):
    """
    Takes an integer t (simplified time units) or pd.Series 't'.

    Args:
    - t: int or pd.series format the simplified time unit.

    Returns:
    - r2: returns int if 't' is int or pd.series if 't' is pd.series
    """
    if type(t) == int:
            
        timestamp = start_timestamp + pd.Timedelta(minutes=t * MINUTE)
        r2 = timestamp.month

    else:
        timedelta_values = pd.to_timedelta(t * MINUTE, unit='m')
        timestamp_val = start_timestamp + timedelta_values
        r2 = timestamp_val.dt.month

    return r2

# Example usage:
t = 948  # Example simplified time unit
print("r0:", get_r0(t))
print("r1:", get_r1(t))
print("r2:", get_r2(t))

r0: 84
r1: 2
r2: 4


## preparing the statistics of empty and freight trains

In [15]:
################################################################################
# The following function takes an dataframe as input and  extracts the traingroup statistics
################################################################################

def get_traingroup_statistics_for_df(sample_df,file_name):
    sample_df = sample_df[sample_df['point_ptcarsequencenumber'] >= 101]
    unique_i_tr_traingroup = sample_df[['i', 'tr_traingroup']].drop_duplicates()

    # Calculate total number of unique trains & unique train groups
    total_trains = unique_i_tr_traingroup['i'].nunique()
    unique_train_group = unique_i_tr_traingroup['tr_traingroup'].unique()

    # Count the number of trains in each group
    group_counts = unique_i_tr_traingroup['tr_traingroup'].value_counts().reset_index()
    group_counts.columns = ['tr_traingroup', 'number_of_trains']

    # create a dataframe
    summ_df = pd.DataFrame(index=['No.of Trains'], columns=unique_train_group)

    # Assign counts to the corresponding columns
    for _, row in group_counts.iterrows():
        summ_df.at['No.of Trains', row['tr_traingroup']] = row['number_of_trains']

    # Add the total number of trains to the DataFrame and make the type to int
    summ_df.at['No.of Trains', 'Total'] = total_trains
    summ_df = summ_df.astype({"Total": int})
    
    summ_df.to_csv(file_name, index=False)
    return summ_df,unique_i_tr_traingroup

summ_df,unique_i_tr_traingroup = get_traingroup_statistics_for_df(effective_df,EFFECTIVE_DF_STATS_FILE_PATH)
summ_df

,FREIGHT,PASSENGER,Total
No.of Trains,3811,5869,9680


### Extracting the Departure and Arrival time of all trains

Extracting the Departure and Arrival time of all trains. This is needed to construct $D_{i,t}$  and $a_{i,t}(w_{i,t})$:
$$
a_{i,t}(w_{i,t})  ~:=~  \text{whether train $i$ is active at time $t$,}
$$
so that
$$
D_{i,t} = \underset{s}{\min } ~ s ~ \text{ subject to } a_{i, s}(0) \geq 1,  ~ r_1(s) = r_1(t),
$$
which implies $a_{i,D_{i,t}}(0) = 1$.

In [16]:
################################################################################
# The following functions uses effective_df with freight and empty trains
# and extracts the departure time 't' and arrival time 't' . 
# The departure time 't' for every train 'i' corresponding to the point_ptcarsequencenumber 101  
# Note: (for some trains 101 is not present so it takes the next available minimum number 
# in the point_ptcarsequencenumber for ex. 102) as departure time 't' train of train 'i'
# and arrival time 't' corresponding to the max point_ptcarsequencenumber for the train 'i' 
# and saves the result to the local file EFFECTIVE_DF_DA_IT_W_FREIGHT_FILE_PATH
#
# The function returns a dataframe with 7 columns:
#                   'i': train,
#                    'start_t': group.loc[start_idx, 't'],
#                    'start_wit': group.loc[start_idx, 'w(i,t)'],
#                    'start_t-wit': start_t_minus_wit,
#                    'start_s': group.loc[start_idx, 's'],
#                    'end_t': group.loc[idx - 1, 't'],
#                    'end_wit': group.loc[idx - 1, 'w(i,t)'],
#                    'end_t-wit': end_t_minus_wit,
#                    'end_s': group.loc[idx - 1, 's'],
#                    'tr_traingroup':  group.loc[start_idx, 'tr_traingroup'], 
#                    'tr_isempty':  group.loc[start_idx, 'tr_isempty']  
### This dataframe stored in the file is used to drop the frieght and empty trains at secondary_delay_df
################################################################################

def create_formatted_data_DA_it():
    # Filter and sort the DataFrame
    temp_effective_df = effective_df[effective_df['point_ptcarsequencenumber'] >= 101]
    temp_effective_df = temp_effective_df.groupby(['i', 't', 's', 'point_ptcarsequencenumber',
                                                   'tr_isempty','tr_traingroup'])['w(i,t)'].max().reset_index()
    temp_effective_df.drop_duplicates(inplace=True)
    temp_effective_df.sort_values(by=['i', 't'], ascending=[True, True], inplace=True)
    
    result_data = []
    
    # Group by 'i'
    for train, group in temp_effective_df.groupby('i'):
        group.sort_values(by=['t','point_ptcarsequencenumber'], ascending=[True, True], inplace=True)
        group = group.reset_index(drop=True)
        
        indices = group.index[group['point_ptcarsequencenumber'] == 101].tolist()

        if not indices:
            print(f"No 101 point_ptcarsequencenumber for train {train}, so using the first available point as start.")

        # Iterate over the group to detect sudden decreases in sequence numbers
        start_idx = 0
        for idx in range(1, len(group)):
            # If decrease detected in the point_ptcarsequencenumber, end the current trip
            if group.loc[idx, 'point_ptcarsequencenumber'] < group.loc[idx - 1, 'point_ptcarsequencenumber']:
                start_t_minus_wit = group.loc[start_idx, 't'] - (group.loc[start_idx, 'w(i,t)'] // (60*5)).astype(T_DATATYPE) 
                end_t_minus_wit = group.loc[idx-1, 't'] - (group.loc[idx-1, 'w(i,t)'] // (60*5)).astype(T_DATATYPE) 
                
                result_data.append({
                    'i': train,
                    'start_t': group.loc[start_idx, 't'],
                    'start_wit': group.loc[start_idx, 'w(i,t)'],
                    'start_t-wit': start_t_minus_wit,
                    'start_s': group.loc[start_idx, 's'],
                    'end_t': group.loc[idx - 1, 't'],
                    'end_wit': group.loc[idx - 1, 'w(i,t)'],
                    'end_t-wit': end_t_minus_wit,
                    'end_s': group.loc[idx - 1, 's'],
                    'tr_traingroup':  group.loc[start_idx, 'tr_traingroup'], 
                    'tr_isempty':  group.loc[start_idx, 'tr_isempty']  
                })
                
                # Start a new trip
                start_idx = idx

        # Extract the last trip
        start_t_minus_wit = group.loc[start_idx, 't'] - (group.loc[start_idx, 'w(i,t)'] // (60*5)).astype(T_DATATYPE) 
        end_t_minus_wit =  group.iloc[-1]['t'] - (group.iloc[-1] ['w(i,t)'] // (60*5)).astype(T_DATATYPE) 
        result_data.append({
            'i': train,
            'start_t': group.loc[start_idx, 't'],
            'start_wit': group.loc[start_idx, 'w(i,t)'],
            'start_t-wit': start_t_minus_wit,
            'start_s': group.loc[start_idx, 's'],
            'end_t': group.iloc[-1]['t'],
            'end_wit': group.iloc[-1]['w(i,t)'],
            'end_t-wit': end_t_minus_wit,
            'end_s': group.iloc[-1]['s'],
            'tr_traingroup':  group.loc[start_idx, 'tr_traingroup'], 
            'tr_isempty':  group.loc[start_idx, 'tr_isempty']  
        })
    
    result_df = pd.DataFrame(result_data).drop_duplicates()
    result_df.to_parquet(EFFECTIVE_DF_DA_IT_W_FREIGHT_FILE_PATH, index=False)

    return result_df

DA_it_df = create_formatted_data_DA_it()
print("\n", DA_it_df.shape)
DA_it_df.head(20)

No 101 point_ptcarsequencenumber for train -99983, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -99981, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -98848, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -97694, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -97065, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -95761, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -95598, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -95125, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -95035, so using the first available point as start.
No 101 point_ptcarsequencenumber for train -94410, so using the first available point as start.
No 101 point_ptcarsequencenumber for tra

,i,start_t,start_wit,start_t-wit,start_s,end_t,end_wit,end_t-wit,end_s,tr_traingroup,tr_isempty
0,-99992,978,9013.36,948,1010,979,8899.40,950,1715,PASSENGER,1
1,-99992,13085,12423.00,13044,1011,13086,12286.00,13046,1715,PASSENGER,1
2,-99991,2175,420.75,2174,175,2178,871.00,2176,848,FREIGHT,0
3,-99991,2456,-1628.00,2462,175,2458,-1335.00,2463,848,FREIGHT,0
4,-99991,4948,-2067.00,4955,1034,4950,-1950.64,4957,848,FREIGHT,0
5,-99991,5236,-2040.00,5243,1034,5238,-1817.64,5245,848,FREIGHT,0
6,-99991,5521,-2900.00,5531,1034,5523,-2776.64,5533,848,FREIGHT,0
7,-99991,5810,-2593.00,5819,1034,5812,-2464.32,5821,848,FREIGHT,0
8,-99991,15951,1163.76,15948,205,15961,1484.92,15957,981,FREIGHT,0
9,-99990,10390,1374.64,10386,417,10390,1331.92,10386,1025,FREIGHT,0


In [17]:
################################################################################
# The following function drops three categories: 
#    - point_ptcarsequencenumber< 101, 
#    - empty trains 
#    - frieght trains 
# 
# It finally generates an external local file in the working directory specified 
# in the EFFECTIVE_DF_CLEANED_FILE_PATH, that contains only the needed columns of effective_df
################################################################################

def store_cleaned_effective_df(effective_df):
    print('Before clean: ', effective_df.shape)
    effective_df = effective_df[
        (effective_df['point_ptcarsequencenumber'] >= 101) &
        (effective_df['tr_isempty'] != 1) &
        (effective_df['tr_traingroup'] != 'FREIGHT')
    ]

    effective_df.drop_duplicates(inplace=True)
    effective_df.reset_index(drop=True, inplace=True)

    print('After clean: ', effective_df.shape)
    
    effective_df.to_parquet(EFFECTIVE_DF_CLEANED_FILE_PATH, index=False)
    return effective_df

effective_df = store_cleaned_effective_df(effective_df)
effective_df.head(2)

Before clean:  (22367075, 19)


C:\Users\w.bi\AppData\Local\Temp\ipykernel_22188\3448232281.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effective_df.drop_duplicates(inplace=True)


After clean:  (18209777, 19)


,i,tr_isempty,tr_traingroup,tr_type,m_org,point_ptcarsequencenumber,point_isunexpected,planned_status,planned_timetabletype,s,"w(i,t)",point_micro_time,point_micro_fromptrefsymbolicnam,day_name,formatted_point_macro_time,r0,r1,r2,t
0,-53372,0,PASSENGER,Ntype,out,101,0,ACTIVE,COMMERCIAL_STOP,219,8.84,2023-04-01 00:02:29+02:00,367064.0,Saturday,2023-04-01 00:02:08+02:00,0,6,4,0
1,-85230,0,PASSENGER,L,in,101,0,ACTIVE,IMPORTANT_PASSAGE,1196,131.00,2023-04-01 00:03:07+02:00,401061.0,Saturday,2023-04-01 00:02:11+02:00,0,6,4,0


In [18]:
DA_it_df = read_function_data(EFFECTIVE_DF_DA_IT_W_FREIGHT_FILE_PATH)

In [19]:
DA_it_df

,i,start_t,start_wit,start_t-wit,start_s,end_t,end_wit,end_t-wit,end_s,tr_traingroup,tr_isempty
0,-99992,978,9013.36,948,1010,979,8899.40,950,1715,PASSENGER,1
1,-99992,13085,12423.00,13044,1011,13086,12286.00,13046,1715,PASSENGER,1
2,-99991,2175,420.75,2174,175,2178,871.00,2176,848,FREIGHT,0
3,-99991,2456,-1628.00,2462,175,2458,-1335.00,2463,848,FREIGHT,0
4,-99991,4948,-2067.00,4955,1034,4950,-1950.64,4957,848,FREIGHT,0
...,...,...,...,...,...,...,...,...,...,...,...
419468,-17,24961,59.60,24961,642,24984,266.72,24984,219,PASSENGER,0
419469,-17,25249,46.60,25249,642,25272,393.72,25271,219,PASSENGER,0
419470,-17,25537,53.60,25537,642,25559,14.72,25559,219,PASSENGER,0
419471,-17,25825,43.60,25825,642,25847,-5.28,25848,219,PASSENGER,0


In [20]:
effective_df = read_function_data(EFFECTIVE_DF_CLEANED_FILE_PATH)

#### Filling the a_it array

Filling a dataframe to fill if a train is active at time 't'

In [10]:
################################################################################
# The following functions uses DA_it_adjusted_df and creates a new dataframe with 
# 't' as row index and train i as column index. The cells contains Boolean values  
# encoding whether train i is active at time t.
# This resulting binary matrix is saved to the local file EFFECTIVE_DF_A_IT_FILE_PATH
################################################################################

def create_formatted_data_a_it(effective_df, DA_it_df, start_t_col, end_t_col, fileName):
    # Filter only passenger trains and non empty trains from DA_it_df    
    D_it_df = DA_it_df[(DA_it_df['tr_traingroup'] == 'PASSENGER') & (DA_it_df['tr_isempty'] == 0)]

    # Fill 't' index with 0 to the max 't' value from effective_df
    max_t = effective_df['t'].max()
    t = np.arange(0, max_t + 1)

    unique_i = D_it_df['i'].unique()

    # Create an empty DataFrame with 't' as index and unique 'i' values as columns
    a_it_df = pd.DataFrame(index=t, columns=unique_i, data=np.zeros((len(t), len(unique_i)), dtype=int))

    # Iterate over each row in D_it_df to populate the a_it_df DataFrame
    for _, row in D_it_df.iterrows():
        # Get indices for the range of 't' values between 'start_t_col' and 'end_t_col'
        #https://numpy.org/doc/stable/reference/generated/numpy.searchsorted.html
        start_idx = np.searchsorted(t, row[start_t_col], side='left')
        end_idx = np.searchsorted(t, row[end_t_col], side='right')

        # Set '1' for 't' values between 'start_t' and 'end_t' for each 'i'
        a_it_df.iloc[start_idx:end_idx, a_it_df.columns.get_loc(row['i'])] = 1

    a_it_df.to_parquet(fileName, index=False)
    
    return a_it_df


a_it_df = create_formatted_data_a_it(effective_df, DA_it_df, 'start_t', 'end_t', EFFECTIVE_DF_A_IT_FILE_PATH)
a_it_w0_df = create_formatted_data_a_it(effective_df, DA_it_df, 'start_t-wit', 'end_t-wit', EFFECTIVE_DF_A_IT_W0_FILE_PATH)

print("\n", a_it_df.shape)
a_it_df.head(10)


 (26209, 5469)


,-99970,-99968,-99967,-99936,-99929,-99927,-99925,-99910,-99879,-99876,...,-240,-218,-200,-198,-97,-88,-85,-82,-79,-17
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Departure of train

Filling a dataframe with earliest deperature of train 'i'

In [13]:
################################################################################
# The following functions uses a_it_df and creates a new dataframe with 
# 't' as row index and train i as column index. The cells 
# contains the earliest departure time of train 'i' or when the train was active from the a_it_df
# This resulting dataframe is saved to the local file EFFECTIVE_DF_D_IT_FILE_PATH
################################################################################

a_it_df = read_function_data(EFFECTIVE_DF_A_IT_FILE_PATH)
a_it_df.columns = a_it_df.columns.astype(int)


def create_formatted_data_D_it(a_it_df):
    """
    Args:
    - a_it_df: The DataFrame with 'a_it' values (train active at each time 't').

    Returns:
    - D_it_df: DataFrame with row index as time (t), columns as train IDs (i), and values as D_it.
    """
    
    # Initialize the DataFrame for D_it_df
    a_it_df.columns = a_it_df.columns.astype(int)
    D_it_df = pd.DataFrame(index=a_it_df.index, columns=a_it_df.columns)
    max_index = a_it_df.index.max()
    
    # Process each train identifier
    for col in a_it_df.columns:
        column_data = a_it_df[col]
        
        # Identify points where the series changed from 0 to 1, earliest departure time of train 'i' 
        trip_starts = (column_data.shift(1, fill_value=0) == 0) & (column_data == 1)
        start_index = column_data.index[trip_starts]
        
        # Identify points after which the series changes from 1 to 0, indicating the last active time of train
        trip_ends = (column_data.shift(-1, fill_value=0) == 0) & (column_data == 1)
        end_index = column_data.index[trip_ends]
        
        ## Fetch all the index at which train stopped
        train_halt_index = end_index + 1
        
        # Filter out invalid indices greater than max_index
        valid_mask = train_halt_index <= max_index
        train_halt_index = train_halt_index[valid_mask]
        #end_index = end_index[valid_mask]
        
        # Initialize the DataFrame to hold the trip times
        trip_df = pd.DataFrame(index=a_it_df.index, columns=[col])
        trip_df[col] = np.nan

        # Assign the earliest departure time of train 'i' and last active time of train
        trip_df.loc[start_index, col] = start_index
        trip_df.loc[train_halt_index, col] = -1

        # Backward fill the trip times
        trip_df.ffill(inplace=True)

        # Assign the trip to D_it_df
        D_it_df[col] = trip_df[col]
        D_it_df[col] = D_it_df[col].fillna(-1).astype(int)
        

    D_it_df.to_parquet(EFFECTIVE_DF_D_IT_FILE_PATH, index=False)
        
        
# Create the optimized D_it_df DataFrame
create_formatted_data_D_it(a_it_df)

KeyboardInterrupt: 

Extracting the earliest deperature of train 'i' at w.r.t time 't'

In [88]:
D_it_df = read_function_data(EFFECTIVE_DF_D_IT_FILE_PATH)
D_it_df.columns = D_it_df.columns.astype(int)

def get_D_it(i, t, row=None, df=D_it_df):
    """
    Check earliest departure time of train 'i' corresponding to time 't'
    
    Args:
    - i: Value for 'i' column (train)
    - t: time 't' 
    - row: pd.Series, optional DataFrame row containing the values for i and t.

    Returns:
    - int: The latest index where a change from 0 to 1 occurred before or at 't', 
            or the latest occurrence of 1 before 't'.
    """
    
    # If a dataframe row is provided as input we extract 'i' and 't' from the row
    if row is not None:
        i = row[i]
        t = row[t]
        
    if(D_it_df[i].iloc[t]==-1):
        return t
    else:
        return D_it_df[i].iloc[t]

# start_t = get_D_it(-43095, 73)
# print("Start Index of 't':", start_t)

# start_t = get_D_it(-43095, 71)
# print("Start Index of 't':", start_t)

# print(get_D_it(-99970, 518))
# print(get_D_it(-99970, 537))
# print(get_D_it(-99970, 538))
# print(get_D_it(-99970, 710))
print(get_D_it(-99970, 809))
# get_D_it(-9003, 105)

808


## EndogenousData class

In [4]:
################################################################################
# The following is a Class named 'EndogenousData' that contains a dataframe with the following columns:
#class EndogenousData:

    # This contains a dataframe with the following columns:
    # - i
    # - j
    # - t
    # - c
    # - w_{i,j,t}^c
    
 # Only in this file 'PrepareDataForPrediction.ipynb' the w_{i,j,t}^c in EndogenousData coincides with the observed data
#############################################################################

class EndogenousData:
    
    def __init__(self, dataframe):
        """
        Initializes the EndogenousData class with a DataFrame.

        :param dataframe: A pandas DataFrame containing columns 'i', 'j', 't', 'c', and 'w_{i,j,t}^c'.
        :raises ValueError: If the DataFrame does not contain the required columns.
        """
        required_columns = {'i', 'j', 't', 'c', 'w_{i,j,t}^c'}
        
        # Check if all required columns are present in the DataFrame
        if not required_columns.issubset(dataframe.columns):
            raise ValueError(f"DataFrame must contain the following columns: {required_columns}")
        
        # Initialize the instance with the DataFrame
        self.dataframe = dataframe
    
    def get_data(self):
        """
        Returns the DataFrame containing the observed data.

        :return: A pandas DataFrame with the data.
        """
        return self.dataframe

    def filter_data(self, i=None, j=None, t=None, c=None):
        """
        Filters the DataFrame based on the provided criteria.

        :param i: Value to filter column 'i'. If None, no filter is applied for 'i'.
        :param j: Value to filter column 'j'. If None, no filter is applied for 'j'.
        :param t: Value to filter column 't'. If None, no filter is applied for 't'.
        :param c: Value to filter column 'c'. If None, no filter is applied for 'c'.

        :return: A pandas DataFrame containing the filtered data.
        """
        df = self.dataframe
        
        # Apply filters based on provided criteria
        if i is not None:
            df = df[df['i'] == i]
        if j is not None:
            df = df[df['j'] == j]
        if t is not None:
            df = df[df['t'] == t]
        if c is not None:
            df = df[df['c'] == c]
        
        return df


## Prepare column for function S in the EFFECTIVE data set

We define $r_0(t)$, $r_1(t)$, and $r_2(t)$ as the respective time of the day (in minutes), the day of the week (from one to seven) and month of the year (from one to seven), corresponding to data time $t$. We introduce the notation $\boldsymbol{r}(t) = (r_0(t), r_1(t), r_2(t))$.

The following code is to create $r_0(t)$, $r_1(t)$, $r_2(t)$ in blocks of 5 minutes, as well as to build function S.

We need fo construct the following function:
$$
S_{i,s}^{\boldsymbol{r}(t)}(w_{i,t}) :=~  \text{whether train $i$ passes through station $s$ at time $r_0(t)$, day $r_1(t)$, month $r_2(t)$ } 
$$
When $w_{i,t}/5$ seconds of delay is experience by train $i$ at clock time $r_0(t)$, the binary indicator of whether train $j$ passes through station $s$ becomes $S_{i,s,r_0(\tilde{t}_{i,t} ), r_1(\tilde{t}_{i,t}), r_2(\tilde{t}_{i,t})}$, where $\tilde{t}_{i,t} = t + w_{i,t}$. 

In [22]:
# filling train type 

# Step 1: for each train i, find a non-Ntype tr_type (if any)
train_type_map = (
    effective_df
    .loc[effective_df['tr_type'] != 'Ntype', ['i', 'tr_type']]
    .drop_duplicates(subset=['i'])
    .set_index('i')['tr_type']
)

# Step 2: replace Ntype using the mapping
effective_df['tr_type'] = np.where(
    effective_df['tr_type'] == 'Ntype',
    effective_df['i'].map(train_type_map),
    effective_df['tr_type']
)

# Optional: if some trains still have no type, keep them as 'Ntype'
effective_df['tr_type'] = effective_df['tr_type'].fillna('Ntype')


In [25]:
################################################################################
# The following functions create_formatted_data_frame_for_S_function creates a formatted dataframe from our effective_df
#  generates external local file FUNCTION_S_R_W0_FILE_PATH in the working directory, 
# which will be used for the function get_S_r,get_S_t, get_S_r_w0,get_S_t_w0
################################################################################

def create_formatted_data_frame_for_S_function():
    """
    Creates formatted data frames from the `effective_df` DataFrame for use in function S.
    
    1. **s_r_w_df**:
       - Groups `effective_df` by columns (`'i'`, `'s'`, `'t'`, `'m'`, `'r0'`, `'r1'`, `'r2'`) and computes the maximum of `'w(i,t)'` for each group.
       - Calculates the adjusted time (`'t-w(i,t)'`) by subtracting the floor division of `'w(i,t)'` by 5-minute blocks (converted to minutes) from `'t'`.
       - Extracts the components (`'r0'`, `'r1'`, `'r2'`) for the adjusted time (`'t-w(i,t)'`) using the previously defined functions 
       `get_r0`, `get_r1`, and `get_r2`.
       - Saves the result to a Parquet file in the working directory specified by `FUNCTION_S_R_W0_FILE_PATH`.
    
    Returns:
    - s_r_w_df: DataFrame with the maximum values of `'w(i,t)'` and adjusted time components.
    """
    
    df = effective_df.copy()
    df['formatted_point_macro_time'] = pd.to_datetime(df['formatted_point_macro_time'])
    
    # Sort by train, station, time, and m_org so IN comes before OUT
    df.sort_values(by=['i','s','formatted_point_macro_time','m_org'], 
                   ascending=[True, True, True, True], inplace=True)
    
    # Add flags
    df['has_in']  = df['m_org'].eq('in')
    df['has_out'] = df['m_org'].eq('out')

    # Initialize 'm' as stop
    df['m'] = 'stop'

    # Identify passing rows: IN immediately followed by OUT with nearly same time
    df['next_m_org'] = df.groupby(['i','s'])['m_org'].shift(-1)
    df['next_time']  = df.groupby(['i','s'])['formatted_point_macro_time'].shift(-1)
    
    # Time difference in seconds
    df['time_diff_sec'] = (df['next_time'] - df['formatted_point_macro_time']).dt.total_seconds().fillna(9999)
    
    # Passing condition: IN followed by OUT within 2 seconds
    passing_mask = (df['m_org']=='in') & (df['next_m_org']=='out') & (df['time_diff_sec']<2)
    
    # Mark both rows as passing
    df.loc[passing_mask, 'm'] = 'passing'
    df.loc[passing_mask.shift(1, fill_value=False), 'm'] = 'passing'  # previous row? optional
    
    # Drop helper columns
    df.drop(columns=['has_in','has_out','next_m_org','next_time','time_diff_sec'], inplace=True)


    # ===== BUILD THE Signal COLUMN =====
    # def pick_signal(g):
    #     """Return signal value based on rules."""
    #     in_vals = g.loc[g['m_org']=='in', ['point_micro_fromptrefsymbolicnam', 'point_micro_time']].dropna()
    #     out_vals = g.loc[g['m_org']=='out', ['point_micro_fromptrefsymbolicnam', 'point_micro_time']].dropna()

    #     if len(in_vals) > 0:
    #         return pd.Series(in_vals.iloc[0].values, index=['Signal', 'Micro_time'])  # rule 1
    #     elif len(out_vals) > 0:
    #         return pd.Series(out_vals.iloc[0].values, index=['Signal', 'Micro_time'])  # rule 2
    #     else:
    #         return pd.Series([np.nan, np.nan], index=['Signal', 'Micro_time'])  # rule 3
            
    # signal_df = df.groupby(['i','s','t']).apply(pick_signal).reset_index()


    def pick_signal(df):
        """
        Return a DataFrame with one row per group (i, s, t), choosing:
        - the first 'in' row if exists
        - otherwise the first 'out' row
        Adds columns 'Signal' and 'Micro_time'.
        """
        # Assign priority: 'in' = 1, 'out' = 2
        df = df.copy()
        df['priority'] = df['m_org'].map({'in': 1, 'out': 2})
    
        # Sort by group and priority
        df_sorted = df.sort_values(['i', 's', 't', 'priority'])
    
        # Take the first row per group
        first_rows = df_sorted.groupby(['i', 's', 't'], as_index=False).first()
    
        # Select and rename columns
        result = first_rows[['i', 's', 't', 'tr_type', 'point_micro_fromptrefsymbolicnam', 'point_micro_time']].rename(
            columns={'point_micro_fromptrefsymbolicnam': 'Signal', 'point_micro_time': 'Micro_time'}
        )
    
        return result
            
    signal_df = pick_signal(df)
    
    # Now you can proceed to build s_r_w_df, grouping and adjusting times as before
    s_r_w_df = df.groupby(['i','s','t','m','r0','r1','r2'])['w(i,t)'].max().reset_index()
    
    s_r_w_df['t-w(i,t)'] = s_r_w_df['t'] - (s_r_w_df['w(i,t)']//(60*5)).astype(T_DATATYPE)
    s_r_w_df['r0(t - max(w(i,t)))'] = get_r0(s_r_w_df['t-w(i,t)'])
    s_r_w_df['r1(t - max(w(i,t)))'] = get_r1(s_r_w_df['t-w(i,t)'])
    s_r_w_df['r2(t - max(w(i,t)))'] = get_r2(s_r_w_df['t-w(i,t)'])
    s_r_w_df['w(i,t)'] = s_r_w_df['w(i,t)'].fillna(0).astype(int)

    # ===== MERGE Signal INTO s_r_w_df =====
    s_r_w_df = s_r_w_df.merge(signal_df, on=['i','s','t'], how='left')
    # s_r_w_df = s_r_w_df.merge(time_df, on=['i','s'], how='left')
    
    s_r_w_df.sort_values('t', inplace=True)
    s_r_w_df.reset_index(drop=True, inplace=True)
    s_r_w_df.to_parquet(FUNCTION_S_R_W0_FILE_PATH, index=False)
    
    print("s_r_w_df shape:", s_r_w_df.shape)
    return s_r_w_df
    
create_formatted_data_frame_for_S_function()

s_r_w_df shape: (10304572, 15)


,i,s,t,m,r0,r1,r2,"w(i,t)","t-w(i,t)","r0(t - max(w(i,t)))","r1(t - max(w(i,t)))","r2(t - max(w(i,t)))",tr_type,Signal,Micro_time
0,-53372,219,0,stop,0,6,4,8,0,0,6,4,IC,367064.0,2023-04-01 00:02:29+02:00
1,-53372,1357,0,passing,0,6,4,-32,1,1,6,4,IC,367064.0,2023-04-01 00:02:29+02:00
2,-85230,1196,0,stop,0,6,4,147,0,0,6,4,L,401061.0,2023-04-01 00:03:07+02:00
3,-85230,1781,0,passing,0,6,4,127,0,0,6,4,L,401061.0,2023-04-01 00:03:07+02:00
4,-53372,1048,1,passing,1,6,4,-18,2,2,6,4,IC,360432.0,2023-04-01 00:08:45+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10304567,-69328,1048,26208,stop,0,6,7,-79,26209,1,6,7,IC,362041.0,2023-06-30 23:59:58+02:00
10304568,-26402,1058,26208,stop,0,6,7,15,26208,0,6,7,IC,501502.0,2023-06-30 23:59:43+02:00
10304569,-57208,553,26208,stop,0,6,7,67,26208,0,6,7,IC,NaN,NaT
10304570,-89150,1047,26208,stop,0,6,7,583,26207,287,5,6,IC,104162.0,2023-06-30 23:59:15+02:00


## Function S in the EFFECTIVE data set

In [27]:
################################################################################
# Call the read_function_data function to read parquet file FUNCTION_S_R_W0_FILE_PATH 
# we have created earlier in the working directory via the function create_formatted_data_frame_for_S_function, 
# to fill data for the dataframes s_r_w_df, which will be used by our 4 different s functions
################################################################################
s_r_w_df = read_function_data(FUNCTION_S_R_W0_FILE_PATH)
# s_r_w_df.head(6)

global S_set_stations 

S_set_stations =  s_r_w_df['s'].unique()
print(len(S_set_stations))
S_set_stations

1041


array([ 219, 1357, 1196, ...,  156, 1936, 1831])

In [28]:
train_type =  s_r_w_df['tr_type'].unique()
print(len(train_type))
train_type

15


array(['IC', 'L', 'CHART', 'PERS', 'ICE', 'TGV', 'THAL', 'EURST', 'ICT',
       'TECH', 'EXT', 'P', 'INT', 'BUS', 'GRP'], dtype=object)

### a_it

In [73]:
a_it_df = read_function_data(EFFECTIVE_DF_A_IT_FILE_PATH)
a_it_df.columns = a_it_df.columns.astype(int)

a_it_w0_df = read_function_data(EFFECTIVE_DF_A_IT_W0_FILE_PATH)
a_it_w0_df.columns = a_it_w0_df.columns.astype(int)


def get_a_it(i, t=None, row=None):
    """
    Check if train 'i' is active at time 't'. If 't' is not provided, return all times 't' where train 'i' is active.

    Parameters:
    - i: Value for 'i' column (train)
    - t: time 't' (optional)
    
    Returns:
    - If 't' is provided: True (1) if train is active at time 't', 0 otherwise.
    - If 't' is not provided: A list or Series of times 't' where train 'i' is active.
    """
    if row is not None:
        i = row[i]
        t = row[t]
        return a_it_df[i].iloc[t]
        
    else:
        if t is not None:
            return a_it_df[i].loc[t]
        else:
            return a_it_df.index[a_it_df[i] == 1].tolist()
    

def get_a_it_w0(i, t=None):
    """
    Check if train 'i' is active at adjusted time 't-w(i,t)'. If 't' is not provided, return all times 't' where train 'i' is active.

    Parameters:
    - i: Value for 'i' column (train)
    - t: adjusted time 't' (optional)
    
    Returns:
    - If 't' is provided: True (1) if train is active at the adjusted time 't-w(i,t)', 0 otherwise.
    - If 't' is not provided: A list or Series of times 't' where train 'i' is active at the adjusted time.
    """
    #if t is not None:
    #return a_it_w0_df[i].loc[t]
    #else:
    return a_it_w0_df.index[a_it_w0_df[i] == 1].tolist()
    
    
    #active_times_i =get_a_it_w0_without_m()
    #active_times_j = a_it_w0_df.index[a_it_w0_df[j] == 1].tolist()   
    #return set(active_times_i.union(active_times_j))
    
    


## These must satisfy get_a_it(i, t - max(w(i,t)),m) = get_a_it_w0(i, t,m)
def get_a_it_with_m(i, t, m):
    """
    Check if train 'i' is active at time 't' on mode 'm'.

    Parameters:
    - i: Value for 'i' column  (train)
    - t: time 't'
    - m: Value for 'm' column 
    
    Returns:
    - True (1) if the combination is present, 0 otherwise
    """
    i_match = s_r_w_df['i'].values == i
    t_match = s_r_w_df['t'].values == t
    m_match = s_r_w_df['m'].values == m
    
    # Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & t_match & m_match
    
    return 1 if np.any(combined_match) else 0

def get_a_it_w0_with_m(i, t, m):
    """
    Check if train 'i' is active at adjusted time 't-w(i,t)'  on mode 'm'.
    
    Parameters:
    - i: Value for 'i' column  (train)
    - t: adjusted time 't'
    - m: Value for 'm' column 
    
    Returns:
    - True (1) if the combination is present, 0 otherwise
    """
    i_match = s_r_w_df['i'].values == i
    t_match = s_r_w_df['t-w(i,t)'].values == t
    m_match = s_r_w_df['m'].values == m
    
    # Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & t_match & m_match
    
    return 1 if np.any(combined_match) else 0

print(get_a_it(-91842, 5))
get_a_it_with_m(-91842, 0,'passing')

0


0

In [26]:
################################################################################
# The following functions `get_S_t`, takes   train 𝑖,  station 𝑠,  time 't', mode 'm' as input 
# verifies the s_r_w_df if the combination is present 
# and returns a boolean if  train 𝑖,  passes thru station 𝑠 at  time 't' 
################################################################################

def get_S_t(i, s, t, m, s_r_w_df):
    """
    Check if the combination of values (i, s, t,m) is present in the s_r_w_df DataFrame.

    Parameters:
    - i: Value for 'i' column  (train)
    - s: Value for 's' column  (station)
    - t: time 't'
    - m: Value for 'm' column 
    - s_r_w_df: DataFrame to check for presence

    Returns:
    - True (1) if the combination is present, 0 otherwise
    """
    # Convert the relevant columns to NumPy arrays for efficient operations and compare
    i_match = s_r_w_df['i'].values == i
    s_match = s_r_w_df['s'].values == s
    t_match = s_r_w_df['t'].values == t
    m_match = s_r_w_df['m'].values == m
    
    # Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & s_match & t_match & m_match
    
    return 1 if np.any(combined_match) else 0

value_to_check = (-99992, 1010,978,'stop')
print(get_S_t(*value_to_check, s_r_w_df))

value_to_check = (-99992, 1011,978,'in')
print(get_S_t(*value_to_check, s_r_w_df))


################################################################################
# The following functions `get_S_r`, takes  train 𝑖,  station 𝑠 , minute 'r0', day 'r1', month 'r2',mode 'm', as input 
# It verifies in the 's_r_w_df' data frame if the combination is present it returns a list {t, get_S_t(i, s, t)} 
# corresponding to r0, r1, r2 
###############################################################################

def get_S_r(i, s, r0, r1, r2, m, s_r_w_df):
    """
    Check if the combination of values (i, s, m, r0, r1, r2) is present in the s_r_w_df DataFrame.

    Parameters:
    - i: Value for 'i' column  (train)
    - s: Value for 's' column  (station)
    - r0: Value for 'r0' column  (minute)
    - r1: Value for 'r1' column  (day)
    - r2: Value for 'r2' column  (month)
    - m: Value for 'm' column
    - s_r_w_df: DataFrame to check for presence

    Returns:
    - return a list {t, get_S_t(i, s, t)} corresponding to r0, r1, r2
    """
    # Convert the relevant columns to NumPy arrays for efficient operations and compare
    i_match = s_r_w_df['i'].values == i
    s_match = s_r_w_df['s'].values == s
    r0_match = s_r_w_df['r0'].values == r0
    r1_match = s_r_w_df['r1'].values == r1
    r2_match = s_r_w_df['r2'].values == r2
    m_match = s_r_w_df['m'].values == m
    
    # Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & s_match & r0_match & r1_match & r2_match & m_match
    combined_time = r0_match & r1_match & r2_match 
    
    values_list = s_r_w_df['t'].values[combined_time].tolist()

    if len(s_r_w_df['t'].values[combined_time].tolist()) > 0: 

        out_list = values_list[0]
        S = 1
    else: 
        
        out_list = []
        S = 0
        
    return out_list, S

value_to_check = (-99992, 1010, 114, 2,4,'stop')
print(get_S_r(*value_to_check, s_r_w_df))

value_to_check = (-99992, 1011,114, 2,4,'passing')
print(get_S_r(*value_to_check, s_r_w_df))


0
0
(978, 1)
(978, 1)


In [13]:
################################################################################
# The following functions `get_S_t`, takes  train 𝑖, station 𝑠 and adjusted time (`'t-w(i,t)'`) as 't',mode 'm', as input
# verifies the s_r_w_df if the combination is present in 'i', 's_vec', 't-w(i-t)'
# and returns a boolean array of 1s and 0s
################################################################################


def get_S_t_w0(i_values, s_values, t_values, m, s_r_w_df, status):
    
    """
    Optimized version of get_S_t_w0 that returns arrays of 0s and 1s for each 't' value
    with 'i' values as columns in the DataFrame.
    
    Parameters:
    - i_values: Array of 'i' values.
    - s_values: Array of 's' values.
    - t_values: Array of 't' values.
    - m: Value for 'm' column.
    - s_r_w_df: DataFrame to check for presence.
    
    Returns:
    - DataFrame with 't' as index, 'i' as columns, and arrays of 0s and 1s as values.
    """
    # Filter the DataFrame for the given 'm' value
    filtered_df = s_r_w_df[s_r_w_df['m'] == m]

    if type(t_values) == int and type(i_values) == int:

        # Create a mask for the conditions that do not depend on s
        i_match = s_r_w_df['i'].values == i_values
        # t_match = s_r_w_df['t-w(i,t)'].values == t_values
        t_match = s_r_w_df['t'].values == t_values
        m_match = s_r_w_df['m'].values == m
        
        combined_mask = i_match & t_match & m_match
        
        # Filter the dataframe based on the combined mask
        filtered_df = s_r_w_df[combined_mask]
    
        # Check for the presence of each s value in the filtered dataframe
        s_present = np.isin(s_values, filtered_df['s'].values)
    
        # Convert boolean array to int (1 for True, 0 for False)
        OUT = s_present.astype(int)

    elif type(t_values) == int and type(i_values) != int:

       # Create an empty DataFrame with 't' as the index and 'i' values as columns
        results_df = pd.DataFrame(index=t_values, columns=i_values)
    
        # Initialize with empty lists that will store arrays for each 't' and 'i'
        for col in results_df.columns:
            results_df[col] = results_df.index.map(lambda _: np.zeros(len(s_values), dtype=np.int8))
    
        # # Group the filtered DataFrame by 'i' and 't-w(i,t)' to avoid looping
        # grouped = filtered_df.groupby(['i', 't-w(i,t)'])

        # Group the filtered DataFrame by 'i' and 't' to avoid looping
        grouped = filtered_df.groupby(['i', 't'])
        
        # Iterate over groups and fill the results DataFrame
        for (i, t), group in grouped:
            if t in t_values:
                # Initialize an array of zeros with the same length as s_values
                present_s = np.zeros(len(s_values), dtype=np.int8)
                
                # Get the 's' values from the group
                s_in_group = group['s'].values
                
                # Find which s_values are present in this group and update present_s
                present_s[np.isin(s_values, s_in_group)] = 1
                
                # Update the corresponding column in the results DataFrame for the given 't' and 'i'
                results_df.at[t, i] = present_s

        OUT = results_df
    
    elif type(t_values) != int and type(i_values) == int:

       # Create an empty DataFrame with 't' as the index and 'i' values as columns
        results_df = pd.DataFrame(index=t_values, columns=i_values)
    
        # Initialize with empty lists that will store arrays for each 't' and 'i'
        for col in results_df.columns:
            results_df[col] = results_df.index.map(lambda _: np.zeros(len(s_values), dtype=np.int8))
    
        # # Group the filtered DataFrame by 'i' and 't-w(i,t)' to avoid looping
        # grouped = filtered_df.groupby(['i', 't-w(i,t)'])

        # Group the filtered DataFrame by 'i' and 't' to avoid looping
        grouped = filtered_df.groupby(['i', 't'])
    
        # Iterate over groups and fill the results DataFrame
        for (i, t), group in grouped:
            if i in i_values:
                # Initialize an array of zeros with the same length as s_values
                present_s = np.zeros(len(s_values), dtype=np.int8)
                
                # Get the 's' values from the group
                s_in_group = group['s'].values
                
                # Find which s_values are present in this group and update present_s
                present_s[np.isin(s_values, s_in_group)] = 1
                
                # Update the corresponding column in the results DataFrame for the given 't' and 'i'
                results_df.at[t, i] = present_s

        OUT = results_df
    
    else:

        # Create maps from values to matrix indices
        t_index = {t: idx for idx, t in enumerate(t_values)}
        i_index = {i: idx for idx, i in enumerate(i_values)}
        s_index = {s: idx for idx, s in enumerate(s_values)}
        
        # Build 3D result array (T × I × S)
        result = np.zeros((len(t_values), len(i_values), len(s_values)), dtype=np.int8)
        
        # Convert t,i,s to index arrays (vectorized)
        if status == 'delay':
            ti = filtered_df['t'].map(t_index)
        else:
            ti = filtered_df['t-w(i,t)'].map(t_index)
        ii = filtered_df['i'].map(i_index)
        si = filtered_df['s'].map(s_index)
        
        # Drop rows where t or i are not in valid ranges
        valid = ti.notna() & ii.notna() & si.notna()
        ti = ti[valid].astype(int)
        ii = ii[valid].astype(int)
        si = si[valid].astype(int)
        
        # Vectorized filling (no loops!)
        result[ti, ii, si] = 1
        
        # Optional: convert to your DataFrame form
        results_df = pd.DataFrame(
            {i: list(result[:, i_index[i], :]) for i in i_values},
            index=t_values
        )

        OUT = results_df
        
    return OUT


################################################################################
# The following function is a helper function to call the 'get_S_t_w0_optimized' function 
################################################################################ 

def get_S_t_w0_optimized_helper(i_values, status):
    
    # t_values = range(s_r_w_df['t-w(i,t)'].min(), s_r_w_df['t-w(i,t)'].max())
    t_values = range(s_r_w_df['t'].min(), s_r_w_df['t'].max())
    m_value = "stop"
    results_stop = get_S_t_w0(i_values, S_set_stations, t_values, m_value, s_r_w_df, status)

    m_value = "passing"
    results_passing = get_S_t_w0(i_values, S_set_stations, t_values, m_value, s_r_w_df, status)
    
    return results_stop,results_passing
    

################################################################################
# The following functions `get_S_r_w0`, takes   train 𝑖,  station 𝑠 and  
# adjusted minute 'r0(t - max(w(i,t)))', adjusted day 'r1(t - max(w(i,t)))', adjusted month 'r0(t - max(w(i,t)))', 
# as (r0, r1, and r2), mode 'm', as input
# verifies s_r_w_df if the combination is is present in 'i', 's', 'r0(t - max(w(i,t)))','r1(t - max(w(i,t)))','r2(t - max(w(i,t)))'
# and returns a list {t, get_S_t_w0(i, s, t)} corresponding to r0, r1, r2 
###############################################################################

def get_S_r_w0(i, s, r0, r1, r2,m, s_r_w_df):
    
    """
    Check if the combination of values (i, s, r0, r1, r2) is present in the s_r_w_df DataFrame columns 
    i, s, adjusted minute r0(t - max(w(i,t)))', adjusted day 'r1(t - max(w(i,t)))', 
    adjusted month 'r2(t - max(w(i,t)))'

    Parameters:
    - i: Value for 'i' column  (train)
    - s: Value for 's' column  (station)
    - r0: Value for adjusted 'r0' column  (minute)
    - r1: Value for adjusted 'r1' column  (day)
    - r2: Value for adjusted 'r2' column  (month)
    - m: Value for 'm' column
    - s_r_w_df: DataFrame to check for presence

    Returns:
    - return a list {t, get_S_t_w0(i, s, t)} corresponding to r0, r1, r2
    """
    # Convert the relevant columns to NumPy arrays for efficient operations and compare
    i_match = s_r_w_df['i'].values == i
    s_match = s_r_w_df['s'].values == s
    r0_match = s_r_w_df['r0(t - max(w(i,t)))'].values == r0
    r1_match = s_r_w_df['r1(t - max(w(i,t)))'].values == r1
    r2_match = s_r_w_df['r2(t - max(w(i,t)))'].values == r2
    m_match = s_r_w_df['m'].values == m
    
    # Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & s_match & r0_match & r1_match & r2_match & m_match
    combined_time = r0_match & r1_match & r2_match 
    
    values_list = s_r_w_df['t-w(i,t)'].values[combined_time].tolist()

    if len(s_r_w_df['t-w(i,t)'].values[combined_time].tolist()) > 0: 

        out_list = values_list[0]
        S = 1
    else: 
        
        out_list = []
        S = 0
        
    return out_list, S



### Construct G functions:
$$
\left\{\begin{array}{ll}
G_{i,j,t,q}^-(W) & = 
\begin{cases}
    \displaystyle \sum_{s \in \mathcal{S}} \sum_{\ell_i = 1}^{\tau} \sum_{\ell_j = 1}^{\tau}  S_{i,s,0}^{\boldsymbol{r}(t-\ell_i)}(w_{i,t-\ell_i}) S_{j,s,0}^{\boldsymbol{r}(t-\ell_j)}(w_{i,t-\ell_j}), & \mbox{ if } q = 0 \\[0.5cm]
    \displaystyle \sum_{s \in \mathcal{S}} \sum_{\ell_i = 1}^{\tau} \sum_{\ell_j = 1}^{\tau}  S_{i,s,1}^{\boldsymbol{r}(t-\ell_i)}(w_{i,t-\ell_i}) S_{j,s,1}^{\boldsymbol{r}(t-\ell_j)}(w_{i,t-\ell_j}), & \mbox{ if } q = 1 \\[0.5cm]  
    \displaystyle \sum_{s \in \mathcal{S}} \sum_{\ell_i = 1}^{\tau} \sum_{\ell_j = 1}^{\tau}  \left[S_{i,s,0}^{\boldsymbol{r}(t-\ell_i)}(w_{i,t-\ell_i}) S_{j,s,1}^{\boldsymbol{r}(t-\ell_j)}(w_{i,t-\ell_j}) + S_{i,s,1}^{\boldsymbol{r}(t-\ell_i)}(w_{i,t-\ell_i}) S_{j,s,0}^{\boldsymbol{r}(t-\ell_j)}(w_{i,t-\ell_j})\right], & \mbox{ if } q = 2 \\[0.5cm]  
\end{cases}  \\[2.0cm]
G_{i,j,t,q}^+(W) & \displaystyle = \sum_{s \in \mathcal{S}} \sum_{\ell_i = 1}^{\tau} \sum_{\ell_j = 1}^{\tau}   S_{i,s, m}^{\boldsymbol{r}(t+\ell_i)}(w_{i,t}) S_{j,s, m}^{\boldsymbol{r}(t+\ell_j)}(w_{j,t}).  
\end{array}\right.
$$
Here $G_{i,j,t}^-$ and $G_{i,j,t}^+$ encodes the degree of space-time proximity between trains $i$ and $j$, based on past and future encounters (resource sharing), respectively.

Based on this characterization of pairwise resourse sharing and potential interference between each pair of trains, the neighborshood list is defined as $\mathcal{E}_{i,t} = \{j \in \mathcal{N} ~:~ \tilde{G}_{i,j,t}^c = 1\}$. We define the whole neighborshood structure as $\mathcal{E}_t = \{(i, j) \in \mathbb{N}^2 ~:~ j \in \mathcal{E}_{i,t}, i \in \mathcal{N}\}$.

In [29]:
##########################################################################################
#
# The following function is the common function for Gplus and Gminus
# i, S_set_stations, time_i, m
# j, S_set_stations, time_i, m
# 
#
##########################################################################################   

def get_G_minus_optimized(i, j, t, τ, h, S_dictionary):
    
    """
    - i (int): Train identifier.
    - j : Delay-causing trains 'j' for a given train 'i'.
    - t : Time.
    - τ : Time window.
    - h : Mode indicator (0, 1, or 2).
    - op : Operation to perform on time ('plus' or 'minus').
    - S_dictionary : dictionary to store previously fetched output for a set of input.
    """

    # Bind dictionary locally for speed
    Sd = S_dictionary

    # Precompute time offsets
    time_range = range(t, t - τ, -1)

    result = 0

    if h == 0 or h == 1:

        m = "stop" if h == 0 else "passing"

        # Precompute (i, time, m) and (j, time, m)
        keys_i = [(i, ti, m) for ti in time_range]
        keys_j = [(j, tj, m) for tj in time_range]

        # Pre-fetch all vectors
        vec_i = [Sd[k] for k in keys_i]
        vec_j = [Sd[k] for k in keys_j]

        # Small τ → tiny double loop = fastest option
        for vi in vec_i:
            for vj in vec_j:
                result += np.dot(vi, vj)

    else:
        # h == 2
        keys_i_stop = [(i, ti, "stop") for ti in time_range]
        keys_i_pass = [(i, ti, "passing") for ti in time_range]
        keys_j_stop = [(j, tj, "stop") for tj in time_range]
        keys_j_pass = [(j, tj, "passing") for tj in time_range]

        i_stop = [Sd[k] for k in keys_i_stop]
        i_pass = [Sd[k] for k in keys_i_pass]
        j_stop = [Sd[k] for k in keys_j_stop]
        j_pass = [Sd[k] for k in keys_j_pass]

        for is0, ip0 in zip(i_stop, i_pass):
            for js0, jp0 in zip(j_stop, j_pass):
                result += np.dot(is0, jp0) + np.dot(ip0, js0)

    return result



def get_G_minus_helper_optimized(i, j, t, delay_dictionary,τ):
   
    m0_minus = get_G_minus_optimized(i, j, t, τ,0,delay_dictionary)
    m1_minus = get_G_minus_optimized(i, j, t, τ,1,delay_dictionary)
    m2_minus = get_G_minus_optimized(i, j, t, τ,2,delay_dictionary)
        
    return m0_minus,m1_minus,m2_minus




In [78]:
##########################################################################################
#
# The following function is the common function for Gplus and Gminus
# i, S_set_stations, time_i, m
# j, S_set_stations, time_i, m
# 
#
##########################################################################################   

def get_G_plus_optimized(i, j, t1, t2, τ, h, S_dictionary):
    
    """
    - i (int): Train identifier.
    - j : Delay-causing trains 'j' for a given train 'i'.
    - t : Time.
    - τ : Time window.
    - h : Mode indicator (0, 1, or 2).
    - op : Operation to perform on time ('plus' or 'minus').
    - S_dictionary : dictionary to store previously fetched output for a set of input.
    """

    # Bind dictionary locally for speed
    Sd = S_dictionary

    # Precompute time offsets
    time_range_i = range(t1, t1 + τ)
    time_range_j = range(t2, t2 + τ)

    result = 0

    if h == 0 or h == 1:

        m = "stop" if h == 0 else "passing"

        # Precompute (i, time, m) and (j, time, m) 
        keys_i = [(i, ti, m) for ti in time_range_i]
        keys_j = [(j, tj, m) for tj in time_range_j]

        # Pre-fetch all vectors
        vec_i = [Sd[k] for k in keys_i]
        vec_j = [Sd[k] for k in keys_j]

        # Small τ → tiny double loop = fastest option
        for vi in vec_i:
            for vj in vec_j:
                result += np.dot(vi, vj)

    else:
        # h == 2
        keys_i_stop = [(i, ti, "stop") for ti in time_range_i]
        keys_i_pass = [(i, ti, "passing") for ti in time_range_i]
        keys_j_stop = [(j, tj, "stop") for tj in time_range_j]
        keys_j_pass = [(j, tj, "passing") for tj in time_range_j]

        i_stop = [Sd[k] for k in keys_i_stop]
        i_pass = [Sd[k] for k in keys_i_pass]
        j_stop = [Sd[k] for k in keys_j_stop]
        j_pass = [Sd[k] for k in keys_j_pass]

        for is0, ip0 in zip(i_stop, i_pass):
            for js0, jp0 in zip(j_stop, j_pass):
                result += np.dot(is0, jp0) + np.dot(ip0, js0)

    return result



def get_G_plus_helper_optimized(i, j, ti, tj,schedule_dictionary,τ):

    m0_plus = get_G_plus_optimized(i, j, ti, tj, τ,0,schedule_dictionary)
    m1_plus = get_G_plus_optimized(i, j, ti, tj, τ,1,schedule_dictionary)
    m2_plus = get_G_plus_optimized(i, j, ti, tj, τ,2,schedule_dictionary)
        
    return m0_plus,m1_plus,m2_plus


In [29]:
##########################################################################################
#
# The following function is the common function for Gplus and Gminus
# Call the get_S_t_w0 function for every combination of 
# i, S_set_stations, time_i, m
# j, S_set_stations, time_i, m
# store the result retrived for every input by keeping the corresponding combination of input 
# (i, time_i, m)   (j, time_j, m) as key 
# and output of get_S_t_w0 as value in delay_dictionary
#
##########################################################################################   

def get_G(i, j, t, τ, h, op, S_dictionary):
    
    """
    - i (int): Train identifier.
    - j : Delay-causing trains 'j' for a given train 'i'.
    - t : Time.
    - τ : Time window.
    - h : Mode indicator (0, 1, or 2).
    - op : Operation to perform on time ('plus' or 'minus').
    - S_dictionary : dictionary to store previously fetched output for a set of input.
    """

    def apply_op(t, li, op):
        return t + li - 1 if op == 'plus' else t - li
        # return t + li if op == 'plus' else t - li
        
    result = 0
    time_range = np.arange(1, τ + 1)
    
    # Compute adjusted times for all li and lj
    times_i = apply_op(t, time_range, op)
    times_j = times_i

    if h == 0 or h == 1:
        m = "stop" if h == 0 else "passing"
        
        ## Call the get_S_t_w0 function for every combination of i, S_set_stations, time_i, m and store in the dictionary
        for time_i in times_i:
            time_i = int(time_i)
            key_i = (i, time_i, m)
            if key_i not in S_dictionary:
                S_dictionary[key_i] = get_S_t_w0(i, S_set_stations, time_i, m, s_r_w_df)
            delay_i = S_dictionary[key_i]
            
            ## Call the get_S_t_w0 function for every combination of j, S_set_stations, time_j, m and store in the dictionary
            for time_j in times_j:
                time_j = int(time_j)
                key_j = (j, time_j, m)
                if key_j not in S_dictionary:
                    S_dictionary[key_j] = get_S_t_w0(j, S_set_stations, time_j, m, s_r_w_df)
                delay_j = S_dictionary[key_j]

                # Multiply the vectors element-wise and sum the result
                product_sum = np.dot(delay_i, delay_j)
                if(product_sum!=0):
                    print(f"T {op} li and t {op} lj", i, j, time_i, time_j, product_sum)
                    result += product_sum
               
    else:
        m = "stop"
        n = "passing"
        
        for time_i in times_i:
            time_i = int(time_i)
            i_m = (i, time_i, m)
            i_n = (i, time_i, n)
                
            # since we have already stored all the combinations in dictionary we dont call
            # get_S_t_w0 again instead, fetch the result from the delay_dictionary
            delay_i0 = S_dictionary[i_m]
            delay_i1 = S_dictionary[i_n]
            
            for time_j in times_j:
                time_j = int(time_j)
                j_m = (j, time_j, m)
                j_n = (j, time_j, n)
                
                delay_j1 = S_dictionary[j_n]
                delay_j0 = S_dictionary[j_m]
                
                # Multiply the vectors element-wise and sum the result
                product_sum = np.dot(delay_i0, delay_j1)
                result += product_sum

                product_sum1 = np.dot(delay_i1, delay_j0)
                result += product_sum1
                    
    return result

# Example usage of get_G:
## Note : for testing purpose i have used different 't' for Gplus and Gminus, usually the same 't' will be passed
# Dictionary to store intermediate results
delay_dict = {}
i= -9001
j = -9002
t = 100

print("passed t", t)
τ = 3  # Example value for τ

m = 0
print("\n Calling G plus ",m)
result = get_G(i, j, t, τ,m,"plus",delay_dict)
print("result = ",result)

m = 1
print("\n Calling G plus ",m)
result = get_G(i, j, t,  τ,m,"plus",delay_dict)
print("result = ",result)

m = 2
print("\n Calling G plus ",m)
result = get_G(i, j, t,  τ,m,"plus",delay_dict)
print("result = ",result)


i= -9001
j = -9002
t = 100
τ = 3  # Example value for τ

m = 0
print("\n Calling G plus ",m)
result = get_G(i, j, t, τ,m,"plus",delay_dict)
print("result = ",result)

m = 1
print("\n Calling G plus ",m)
result = get_G(i, j, t,  τ,m,"plus",delay_dict)
print("result = ",result)

m = 2
print("\n Calling G plus ",m)
result = get_G(i, j, t,  τ,m,"plus",delay_dict)
print("result = ",result)

m = 0
print("\n Calling G minus ",m)
result = get_G(i, j, t, τ,m,"minus",delay_dict)
print("result = ",result)

m = 1
print("\n Calling G minus ",m)
result = get_G(i, j, t, τ,m,"minus",delay_dict)
print("result = ",result)

m = 2
print("\n Calling G minus ",m)
result = get_G(i, j, t, τ,m,"minus",delay_dict)
print("result = ",result)

passed t 100

 Calling G plus  0
result =  0

 Calling G plus  1
result =  0

 Calling G plus  2
result =  0

 Calling G plus  0
result =  0

 Calling G plus  1
result =  0

 Calling G plus  2
result =  0

 Calling G minus  0
result =  0

 Calling G minus  1
result =  0

 Calling G minus  2
result =  0


# Explore and check the correctness of juftif data

In [34]:
# justif_df_org = pd.read_csv(JUSTIF_DATA_FILE_PATH,low_memory=False, sep=';')
justif_df_org = pd.read_csv(JUSTIF_DATA_FILE_PATH,low_memory=False)

# Print the list of columns in the JUSTIF data
print(justif_df_org.columns)

# Print the first two rows in the JUSTIF data
justif_df_org.head(2)

Index(['messageID', 'messageTimestamp', 'tableType', 'justificationID',
       'departuredate', 'trainnumber', 'circulationType', 'automatic',
       'locationptcarid', 'detectiontimestamp', 'ptrefid',
       'delayjustifiedbyreason', 'secondaryReasonID',
       'causingTraindepartureDate', 'causingtrainnumber',
       'causingTraincirculationType', 'signalCausingTrainptrefID',
       'locationCausingTrainptcarID', 'category', 'rubric'],
      dtype='object')


,messageID,messageTimestamp,tableType,justificationID,departuredate,trainnumber,circulationType,automatic,locationptcarid,detectiontimestamp,ptrefid,delayjustifiedbyreason,secondaryReasonID,causingTraindepartureDate,causingtrainnumber,causingTraincirculationType,signalCausingTrainptrefID,locationCausingTrainptcarID,category,rubric
0,d4a62373-fc4c-4c06-992b-e5d25e502580,2023-04-19T23:52:32.043+02:00,primaryJustification,e0c0d24a-c4af-4b96-8a6d-d7030d48904c,2023/4/19,-16346,EMPTY,False,1720,2023-04-19T23:23:16+02:00,102091.0,919,NaN,NaN,NaN,NaN,NaN,NaN,175.0,1439.0
1,9f5a20a0-926f-4647-bfc9-0d73872be50f,2023-04-04T17:40:49.819+02:00,primaryJustification,1fa72c40-e718-49bd-ae39-bcc1de4ef70a,2023/4/4,-63975,COMMERCIAL,False,1271,2023-04-04T17:03:49+02:00,517092.0,4724,NaN,NaN,NaN,NaN,NaN,NaN,175.0,1439.0


In [35]:
################################################################################
# Function to do inital prepocessing of justif data
# Convert 'detectiontimestamp',messageTimestamp to datetime formatting 
# Convert 'trainnumber',locationptcarid to int
# Sorting the converted dataframe in ascending order of detectiontimestamp
################################################################################

def preprocess_justif(temp_df):
    
    '''
    Function to preprocess and clean the justif data
    '''
    # Convert 'detectiontimestamp',messageTimestamp to datetime format
    temp_df['detectiontimestamp'] = pd.to_datetime(temp_df['detectiontimestamp'])
    temp_df['messageTimestamp'] = pd.to_datetime(temp_df['messageTimestamp'])
    
    # Convert 'tr_bk_number' and 'point_ptcarid' to integer format
    temp_df['trainnumber'] = temp_df['trainnumber'].astype(int)
    temp_df['locationptcarid'] = temp_df['locationptcarid'].astype(int)
    
    temp_df = temp_df.sort_values(by='detectiontimestamp')
    return temp_df

justif_df_org= preprocess_justif(justif_df_org)
justif_df_org.head(2)

,messageID,messageTimestamp,tableType,justificationID,departuredate,trainnumber,circulationType,automatic,locationptcarid,detectiontimestamp,ptrefid,delayjustifiedbyreason,secondaryReasonID,causingTraindepartureDate,causingtrainnumber,causingTraincirculationType,signalCausingTrainptrefID,locationCausingTrainptcarID,category,rubric
66268,455e2081-7334-48c1-8348-c904b449ea66,2023-04-01 00:14:55.672000+02:00,primaryJustification,d77bc687-6b2a-449b-8b53-14986bc3afd0,2023/4/1,-85230,COMMERCIAL,False,1195,2023-04-01 00:03:07+02:00,401061.0,189,NaN,NaN,NaN,NaN,NaN,NaN,160.0,1320.0
293273,8bf42954-bc4d-497f-bbc6-a2d478a80494,2023-04-01 00:37:42.250000+02:00,secondaryJustification,ab92dc6d-5728-43f3-9f0d-1b6585d39577,2023/4/1,-85230,COMMERCIAL,False,208,2023-04-01 00:15:35+02:00,402951.0,133,1137.0,2023/4/1,-86432.0,COMMERCIAL,NaN,NaN,NaN,NaN


In [36]:
################################################################################
# Function to Extract strting time of our data from the column detectiontimestamp in justif data
#  The function Sets the global variable 'justif_start_timestamp' to the starting timestamp extracted from the DataFrame

# 'detectiontimestamp' is the times when the delay has happened 
################################################################################

def extract_start_timestamp_from_justif(df,time_column):
    
    """
    Extracts the starting timestamp from the specified column in the DataFrame.
    The starting timestamp is set as a global variable 'start_timestamp' for
    accessibility from anywhere after calling this function..

    Args:
    - df: pandas DataFrame containing the timestamps.
    - time_column: str, the name of the column containing timestamps in ISO 8601 format.

    Returns:
    - pd.Timestamp: The starting timestamp extracted from the DataFrame.
    """
    
    # Convert the time_column to datetime if not done earlier
    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        df[time_column] = pd.to_datetime(df[time_column])

    # Sort the DataFrame by the 'timestamp' column
    df_sorted = df.sort_values(by=time_column)

    # Get the date part of the first timestamp
    first_day = df_sorted[time_column].iloc[0].normalize()

    #Set the global variable 'start_timestamp' to the starting timestamp extracted from the DataFrame
    global justif_start_timestamp
    justif_start_timestamp = first_day + pd.Timedelta(hours=0)

    return justif_start_timestamp


extract_start_timestamp_from_justif(justif_df_org,'detectiontimestamp')
justif_start_timestamp

Timestamp('2023-04-01 00:00:00+0200', tz='UTC+02:00')

In [37]:
################################################################################
# This function takes timestamp column as input  and returns 't' (in blocks of 5 minutes) 
# by removing seconds and timezone. For ex. we can pass the columns 'messageTimestamp' 
# and 'detectiontimestamp' in the justif data, as input to the function.
#
# Note: 
#
# 'messageTimestamp' is the times in which the message has been received
# 'detectiontimestamp' is the times when the delay has happened
################################################################################

# justif_df_org['t_5min_message'] = Append_t(justif_df_org['messageTimestamp'])
# justif_df_org['t_5min_detection'] = Append_t(justif_df_org['detectiontimestamp'])

diff_in_minutes_justif= ((justif_df_org['detectiontimestamp'] - justif_start_timestamp).dt.total_seconds() / 60).round(2)

# Divide the difference by 5 to get the running number in minutes
justif_df_org['t_5min_detection'] = (diff_in_minutes_justif// MINUTE).astype(T_DATATYPE) 

justif_df_org.head(5)

,messageID,messageTimestamp,tableType,justificationID,departuredate,trainnumber,circulationType,automatic,locationptcarid,detectiontimestamp,...,delayjustifiedbyreason,secondaryReasonID,causingTraindepartureDate,causingtrainnumber,causingTraincirculationType,signalCausingTrainptrefID,locationCausingTrainptcarID,category,rubric,t_5min_detection
66268,455e2081-7334-48c1-8348-c904b449ea66,2023-04-01 00:14:55.672000+02:00,primaryJustification,d77bc687-6b2a-449b-8b53-14986bc3afd0,2023/4/1,-85230,COMMERCIAL,False,1195,2023-04-01 00:03:07+02:00,...,189,NaN,NaN,NaN,NaN,NaN,NaN,160.0,1320.0,0
293273,8bf42954-bc4d-497f-bbc6-a2d478a80494,2023-04-01 00:37:42.250000+02:00,secondaryJustification,ab92dc6d-5728-43f3-9f0d-1b6585d39577,2023/4/1,-85230,COMMERCIAL,False,208,2023-04-01 00:15:35+02:00,...,133,1137.0,2023/4/1,-86432.0,COMMERCIAL,NaN,NaN,NaN,NaN,3
46482,03518c8b-9b06-46ba-91cc-33c9c8347216,2023-04-01 00:40:53.161000+02:00,primaryJustification,503ac7f2-a332-4fd1-9cd1-1fc94f7d66d2,2023/4/1,-20942,EMPTY,False,1271,2023-04-01 00:21:30+02:00,...,388,NaN,NaN,NaN,NaN,NaN,NaN,175.0,1439.0,4
263343,847ef063-1f6d-4d98-8c79-0537afa03314,2023-04-01 00:31:55.193000+02:00,secondaryJustification,8c6a3d61-10e7-4f84-8409-3c6023c010ce,2023/4/1,-50429,EMPTY,False,272,2023-04-01 00:26:14+02:00,...,112,1135.0,2023/4/1,-50429.0,COMMERCIAL,NaN,NaN,NaN,NaN,5
40953,f0008cf8-9ff2-4429-a17d-34fbb5dabcce,2023-04-01 00:34:50.514000+02:00,primaryJustification,030a0e2d-3e84-43c1-b720-6e95b575da5c,2023/4/1,-37031,COMMERCIAL,False,197,2023-04-01 00:28:38+02:00,...,173,NaN,NaN,NaN,NaN,NaN,NaN,160.0,1320.0,5


## Prepare timestamp column for delay in the JUSTIF data set

We define $r_0(t)$, $r_1(t)$, and $r_2(t)$ as the respective time of the day (in minutes), the day of the week (from one to seven) and month of the year (from one to seven), corresponding to data time $t$. We introduce the notation $\boldsymbol{r}(t) = (r_0(t), r_1(t), r_2(t))$.

The following code is to create $r_0(t)$ in blocks of 5 minutes.

In [40]:
################################################################################
# Function to create r0, r1 and r2 by removing seconds and timezone from the  
# columns 'messageTimestamp' and 'detectiontimestamp' in the justif data
#
# Note: 
# 'messageTimestamp' is the times in which the message has been received
# 'detectiontimestamp' is the times when the delay has happened
################################################################################

r0, r1, r2 = Append_r0_r1_r2(justif_df_org, 'messageTimestamp')
justif_df_org['r0_5min_message'] = r0
justif_df_org['r1_5min_message'] = r1
justif_df_org['r2_5min_message'] = r2

r0,r1,r2 = Append_r0_r1_r2(justif_df_org,'detectiontimestamp')
justif_df_org['r0_5min_detection'] = r0
justif_df_org['r1_5min_detection'] = r1
justif_df_org['r2_5min_detection'] = r2


In [41]:
justif_df_org

,messageID,messageTimestamp,tableType,justificationID,departuredate,trainnumber,circulationType,automatic,locationptcarid,detectiontimestamp,...,locationCausingTrainptcarID,category,rubric,t_5min_detection,r0_5min_message,r1_5min_message,r2_5min_message,r0_5min_detection,r1_5min_detection,r2_5min_detection
66268,455e2081-7334-48c1-8348-c904b449ea66,2023-04-01 00:14:55.672000+02:00,primaryJustification,d77bc687-6b2a-449b-8b53-14986bc3afd0,2023/4/1,-85230,COMMERCIAL,False,1195,2023-04-01 00:03:07+02:00,...,NaN,160.0,1320.0,0,2,6,4,0,6,4
293273,8bf42954-bc4d-497f-bbc6-a2d478a80494,2023-04-01 00:37:42.250000+02:00,secondaryJustification,ab92dc6d-5728-43f3-9f0d-1b6585d39577,2023/4/1,-85230,COMMERCIAL,False,208,2023-04-01 00:15:35+02:00,...,NaN,NaN,NaN,3,7,6,4,3,6,4
46482,03518c8b-9b06-46ba-91cc-33c9c8347216,2023-04-01 00:40:53.161000+02:00,primaryJustification,503ac7f2-a332-4fd1-9cd1-1fc94f7d66d2,2023/4/1,-20942,EMPTY,False,1271,2023-04-01 00:21:30+02:00,...,NaN,175.0,1439.0,4,8,6,4,4,6,4
263343,847ef063-1f6d-4d98-8c79-0537afa03314,2023-04-01 00:31:55.193000+02:00,secondaryJustification,8c6a3d61-10e7-4f84-8409-3c6023c010ce,2023/4/1,-50429,EMPTY,False,272,2023-04-01 00:26:14+02:00,...,NaN,NaN,NaN,5,6,6,4,5,6,4
40953,f0008cf8-9ff2-4429-a17d-34fbb5dabcce,2023-04-01 00:34:50.514000+02:00,primaryJustification,030a0e2d-3e84-43c1-b720-6e95b575da5c,2023/4/1,-37031,COMMERCIAL,False,197,2023-04-01 00:28:38+02:00,...,NaN,160.0,1320.0,5,6,6,4,5,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55063,72615eaa-bcd0-4670-baa6-611790933d9c,2023-06-30 07:28:21.380000+02:00,primaryJustification,22cff58d-93c3-4994-bd98-63a68f94340c,2023/6/29,-46584,COMMERCIAL,False,119,2023-06-30 06:45:04+02:00,...,NaN,172.0,1416.0,26001,89,5,6,81,5,6
50805,8ffe94a6-a77b-4042-99f6-a49df1cb1b3a,2023-06-30 07:04:08.910000+02:00,primaryJustification,9b48a5f2-2fe3-4bac-9303-767a1d2448b7,2023/6/29,-33890,COMMERCIAL,False,456,2023-06-30 06:59:29+02:00,...,NaN,175.0,1439.0,26003,84,5,6,83,5,6
55047,8e17fa8c-1fe5-40ea-8481-38426ea545d0,2023-06-30 08:45:56.740000+02:00,primaryJustification,fbef1bdf-1449-46e1-82e9-e0db6069a938,2023/6/29,-46584,COMMERCIAL,False,638,2023-06-30 08:33:21+02:00,...,NaN,175.0,1437.0,26022,105,5,6,102,5,6
13769,444deb94-38de-4d9e-9b10-204f556bea89,2023-06-30 09:48:43.854000+02:00,primaryJustification,54e51c10-dab2-45f9-aaf1-474ba4043123,2023/6/29,-46584,COMMERCIAL,False,1246,2023-06-30 08:56:52+02:00,...,NaN,161.0,1338.0,26027,117,5,6,107,5,6


In [42]:
################################################################################
# This function adds a new column to df by converting the timestamps (column time_col) 
# into the format hh:mm (column new_time_col)
################################################################################

def Append_r0_hhmm(df, time_col, new_time_col):
    
    """   
    Input Args:
    - df: pandas DataFrame containing the timestamps.
    - time_column: the name of the column containing timestamps in ISO 8601 format.
    - new_col: the name of the column that this function is going to append to df

    Output Returns:
    - A pandas DataFrame contructed from df, with an additional column 'simplified_t' containing hh:mm.

    """
    
    # Convert the time_column to datetime if not done earlier
    if not pd.api.types.is_datetime64_any_dtype(df[time_col]):
        df[time_column] = pd.to_datetime(df[time_col])
    
    # Function to adjust individual timestamps
    def adjust_timestamp(ts):
        if ts.hour == 24 and ts.minute == 0:
            # Extract date and increment by one day
            new_date = ts + pd.Timedelta(days=1)
            # Replace '24:00' with '00:00' of the next day
            new_ts = new_date.replace(hour=0, minute=0, second=0)
            return new_ts
        return ts

    # Apply the adjustment function
    df[new_time_col] = df[time_col].apply(adjust_timestamp)

    # Round to the nearest 5 minutes
    df[new_time_col] = df[new_time_col].dt.round(TIME_FREQUENCY)

    # Format to keep only date, hour, and minute or hour and minute
    df[new_time_col] = df[new_time_col].dt.strftime(TIME_FORMAT)
    
    return df

Append_r0_hhmm(justif_df_org, 'messageTimestamp','r0_hhmm_5min_message')
Append_r0_hhmm(justif_df_org, 'detectiontimestamp','r0_hhmm_5min_detection')


,messageID,messageTimestamp,tableType,justificationID,departuredate,trainnumber,circulationType,automatic,locationptcarid,detectiontimestamp,...,rubric,t_5min_detection,r0_5min_message,r1_5min_message,r2_5min_message,r0_5min_detection,r1_5min_detection,r2_5min_detection,r0_hhmm_5min_message,r0_hhmm_5min_detection
66268,455e2081-7334-48c1-8348-c904b449ea66,2023-04-01 00:14:55.672000+02:00,primaryJustification,d77bc687-6b2a-449b-8b53-14986bc3afd0,2023/4/1,-85230,COMMERCIAL,False,1195,2023-04-01 00:03:07+02:00,...,1320.0,0,2,6,4,0,6,4,00:15,00:05
293273,8bf42954-bc4d-497f-bbc6-a2d478a80494,2023-04-01 00:37:42.250000+02:00,secondaryJustification,ab92dc6d-5728-43f3-9f0d-1b6585d39577,2023/4/1,-85230,COMMERCIAL,False,208,2023-04-01 00:15:35+02:00,...,NaN,3,7,6,4,3,6,4,00:40,00:15
46482,03518c8b-9b06-46ba-91cc-33c9c8347216,2023-04-01 00:40:53.161000+02:00,primaryJustification,503ac7f2-a332-4fd1-9cd1-1fc94f7d66d2,2023/4/1,-20942,EMPTY,False,1271,2023-04-01 00:21:30+02:00,...,1439.0,4,8,6,4,4,6,4,00:40,00:20
263343,847ef063-1f6d-4d98-8c79-0537afa03314,2023-04-01 00:31:55.193000+02:00,secondaryJustification,8c6a3d61-10e7-4f84-8409-3c6023c010ce,2023/4/1,-50429,EMPTY,False,272,2023-04-01 00:26:14+02:00,...,NaN,5,6,6,4,5,6,4,00:30,00:25
40953,f0008cf8-9ff2-4429-a17d-34fbb5dabcce,2023-04-01 00:34:50.514000+02:00,primaryJustification,030a0e2d-3e84-43c1-b720-6e95b575da5c,2023/4/1,-37031,COMMERCIAL,False,197,2023-04-01 00:28:38+02:00,...,1320.0,5,6,6,4,5,6,4,00:35,00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55063,72615eaa-bcd0-4670-baa6-611790933d9c,2023-06-30 07:28:21.380000+02:00,primaryJustification,22cff58d-93c3-4994-bd98-63a68f94340c,2023/6/29,-46584,COMMERCIAL,False,119,2023-06-30 06:45:04+02:00,...,1416.0,26001,89,5,6,81,5,6,07:30,06:45
50805,8ffe94a6-a77b-4042-99f6-a49df1cb1b3a,2023-06-30 07:04:08.910000+02:00,primaryJustification,9b48a5f2-2fe3-4bac-9303-767a1d2448b7,2023/6/29,-33890,COMMERCIAL,False,456,2023-06-30 06:59:29+02:00,...,1439.0,26003,84,5,6,83,5,6,07:05,07:00
55047,8e17fa8c-1fe5-40ea-8481-38426ea545d0,2023-06-30 08:45:56.740000+02:00,primaryJustification,fbef1bdf-1449-46e1-82e9-e0db6069a938,2023/6/29,-46584,COMMERCIAL,False,638,2023-06-30 08:33:21+02:00,...,1437.0,26022,105,5,6,102,5,6,08:45,08:35
13769,444deb94-38de-4d9e-9b10-204f556bea89,2023-06-30 09:48:43.854000+02:00,primaryJustification,54e51c10-dab2-45f9-aaf1-474ba4043123,2023/6/29,-46584,COMMERCIAL,False,1246,2023-06-30 08:56:52+02:00,...,1338.0,26027,117,5,6,107,5,6,09:50,08:55


## Create a reduced Data Frame containing only the necessary columns for the primary and secondary delay

In [43]:
################################################################################
# This function creates a reduced Data Frame containing only the necessary 
# columns for the primary and secondary delay
#
# Note: 
#
# we report both r_5min_message and r_5min_detection. However, the 
# statistical analysis will be based exclusively on r_5min_detection
################################################################################

def extract_needed_delay_cols():
    
    columns_to_keep = ['trainnumber', 'causingtrainnumber', 'secondaryReasonID', 'r0_hhmm_5min_detection',
                       'r0_5min_message','r0_5min_detection', 'r1_5min_message','r1_5min_detection', 'r2_5min_message',
                       'r2_5min_detection', 
                       't_5min_detection','locationptcarid', 'justificationID', 'delayjustifiedbyreason', 
                       'tableType','circulationType']

    # Select only the desired columns for our primary and secondary delay data
    justif_df = justif_df_org.copy()
    justif_df = justif_df.loc[:, columns_to_keep]
    
    ## Rename the cols to suitable for our model
    justif_df.rename(columns={'trainnumber': 'i', 'causingtrainnumber': 'j', 'secondaryReasonID': 'c', 't_5min_detection': 't'}, inplace=True)
    
    """
    combining circulationType and trainnumber and create 2 columns i_extended and j_extended, 
    to address the issue train number and causing train number same .
    """
    
    justif_df['i_extended'] = justif_df['i'].astype(str) +"_"+ justif_df['circulationType']
    justif_df['j_extended'] = justif_df['j'].astype(str) +"_"+ justif_df['circulationType']
    
    justif_df['delayjustifiedbyreason_min'] = ((justif_df['delayjustifiedbyreason'] ) / (60*MINUTE)).astype(int)
    
    print(justif_df.shape)
    justif_df.head(2)
    
    return justif_df

# New reduced Data Frame
justif_df = extract_needed_delay_cols()

justif_df.head(5)

(295402, 19)


,i,j,c,r0_hhmm_5min_detection,r0_5min_message,r0_5min_detection,r1_5min_message,r1_5min_detection,r2_5min_message,r2_5min_detection,t,locationptcarid,justificationID,delayjustifiedbyreason,tableType,circulationType,i_extended,j_extended,delayjustifiedbyreason_min
66268,-85230,NaN,NaN,00:05,2,0,6,6,4,4,0,1195,d77bc687-6b2a-449b-8b53-14986bc3afd0,189,primaryJustification,COMMERCIAL,-85230_COMMERCIAL,nan_COMMERCIAL,0
293273,-85230,-86432.0,1137.0,00:15,7,3,6,6,4,4,3,208,ab92dc6d-5728-43f3-9f0d-1b6585d39577,133,secondaryJustification,COMMERCIAL,-85230_COMMERCIAL,-86432.0_COMMERCIAL,0
46482,-20942,NaN,NaN,00:20,8,4,6,6,4,4,4,1271,503ac7f2-a332-4fd1-9cd1-1fc94f7d66d2,388,primaryJustification,EMPTY,-20942_EMPTY,nan_EMPTY,1
263343,-50429,-50429.0,1135.0,00:25,6,5,6,6,4,4,5,272,8c6a3d61-10e7-4f84-8409-3c6023c010ce,112,secondaryJustification,EMPTY,-50429_EMPTY,-50429.0_EMPTY,0
40953,-37031,NaN,NaN,00:30,6,5,6,6,4,4,5,197,030a0e2d-3e84-43c1-b720-6e95b575da5c,173,primaryJustification,COMMERCIAL,-37031_COMMERCIAL,nan_COMMERCIAL,0


### Extract primary delay data

In [44]:
################################################################################
# This function generates an external local file in the working directory, 
# containing only the primary delay
################################################################################
pd.set_option('mode.chained_assignment', None)
def prepare_primary_delay_df():
    """
    Extract rows from justif df where tableType is primaryJustification.
    """
    temp_df = justif_df[justif_df['tableType']=='primaryJustification']
    
    #sort data by 't' and reset index and save the delay data to the file
    temp_df.sort_values(by='t', inplace=True)
    
    grouped = temp_df.groupby(['i', 't', 'r0_5min_detection', 'r1_5min_detection', 'r2_5min_detection'])['delayjustifiedbyreason'].max().reset_index()
    
    grouped.reset_index(drop=True, inplace=True)
    grouped.to_parquet(PRIMARY_DELAY_FILE_PATH, index=False)
    return grouped
    
pri_delay_df = prepare_primary_delay_df()

print("File name: ", PRIMARY_DELAY_FILE_PATH)

print("\n", pri_delay_df.shape)
pri_delay_df.head(5)


File name:  C:\data\Data_for_primary_delay_5min.parquet

 (226791, 6)


,i,t,r0_5min_detection,r1_5min_detection,r2_5min_detection,delayjustifiedbyreason
0,-99992,978,114,2,4,9048
1,-99992,13085,125,2,5,12423
2,-99991,2176,160,6,4,651
3,-99991,2178,162,6,4,201
4,-99991,15954,114,5,5,127


### Extract secondary delay data

In [45]:
################################################################################
# This function generates an external local file in the working directory, 
# containing only the raw data of secondary delay
################################################################################
pd.set_option('mode.chained_assignment', None)

def prepare_sec_delay_df():
    """
    Extract rows from justif df where tableType is secondaryJustification.
    """
    sec_delay_df = justif_df[justif_df['tableType']=='secondaryJustification']

    #Converting float value of column 'j', column 'c' to int
    sec_delay_df['c'] = pd.to_numeric(sec_delay_df['c'], errors='coerce').astype('Int64')
    sec_delay_df['j']  = pd.to_numeric(sec_delay_df['j'], errors='coerce').astype('Int64')
    
    print(sec_delay_df.shape)
    ## We drop the rows that has same delay causing trains 'j' and 'i'
    sec_delay_df = sec_delay_df[sec_delay_df['i'] != sec_delay_df['j']]
    
    ## We only need these columns
    sec_delay_df = sec_delay_df[['i', 'j', 't', 'c','delayjustifiedbyreason',
                                 'r0_5min_detection','r1_5min_detection','r2_5min_detection']]
    sec_delay_df.drop_duplicates(inplace=True)
    sec_delay_df.reset_index(drop=True, inplace=True)
    
    #save the raw secondary delay data to the file
    sec_delay_df.to_parquet(SECONDARY_DELAY_FILE_PATH, index=False)
    
    return sec_delay_df
    

sec_delay_df = prepare_sec_delay_df()
print("File name: ", SECONDARY_DELAY_FILE_PATH)
print("\n",sec_delay_df.shape)
sec_delay_df.head(2)



(42771, 19)
File name:  C:\data\Data_for_secondary_delay_5min.parquet

 (41229, 8)


,i,j,t,c,delayjustifiedbyreason,r0_5min_detection,r1_5min_detection,r2_5min_detection
0,-85230,-86432,3,1137,133,3,6,4
1,-69328,-29553,8,1137,282,8,6,4


In [46]:
sec_delay_df

,i,j,t,c,delayjustifiedbyreason,r0_5min_detection,r1_5min_detection,r2_5min_detection
0,-85230,-86432,3,1137,133,3,6,4
1,-69328,-29553,8,1137,282,8,6,4
2,-50429,-33673,9,1134,154,9,6,4
3,-24885,-66586,11,1137,184,11,6,4
4,-17800,-19814,12,1137,400,12,6,4
...,...,...,...,...,...,...,...,...
41224,-86289,-51298,8636,1137,126,284,7,4
41225,-26325,-19172,8637,1137,126,285,7,4
41226,-26325,-47506,8637,1134,92,285,7,4
41227,-47290,-26325,8638,1137,452,286,7,4


### Mark empty / freight trains on primary/secondary delay df 

In [47]:
################################################################################
# The following functions get_tr_traingroup_empty_status, get_empty_freight_trains uses DA_it_df  
# takes 'i' from pri_delay_df finds  traingroup of 'i' and empty or non empty status of 'i' 
# creates new columns in pri_delay_df namely i_traingroup, i_empty
# (or)
#  takes 'i','j' from sec_delay_df finds traingroup of 'i' 'j', and empty or non empty status of 'i' and 'j'
# creates new columns in sec_delay_df namely i_traingroup, j_traingroup,i_empty,j_empty
################################################################################

DA_it_df = read_function_data(EFFECTIVE_DF_DA_IT_W_FREIGHT_FILE_PATH)
DA_it_df['tr_isempty_encoded'] = DA_it_df['tr_isempty'].replace({0: 'Not Empty', 1: 'Empty'})

def get_tr_traingroup_empty_status(row, col_name, col_to_extract):
    
    # Filter DA_it_df based on the conditions
    matching_rows = DA_it_df.loc[
        (DA_it_df['i'] == row[col_name]) & 
        (row['t'] >= DA_it_df['start_t']) & 
        (row['t'] <= DA_it_df['end_t']), 
        col_to_extract
    ]
    
    # Check if there's any matching row
    if not matching_rows.empty:
        return matching_rows.values[0]
    else:
        # Return None or an empty Series if no match is found
        return None
    
     
def get_empty_freight_trains(new_df):
    # Apply the function to each row in new_df for the 'i' column, 'j' column
    new_df['i_empty'] = new_df.apply(lambda row: get_tr_traingroup_empty_status(row, 'i','tr_isempty_encoded'), axis=1)
    new_df['i_traingroup'] = new_df.apply(lambda row: get_tr_traingroup_empty_status(row, 'i', 'tr_traingroup'), axis=1)
    
    if 'j' in new_df.columns:
        new_df['j_empty'] = new_df.apply(lambda row: get_tr_traingroup_empty_status(row, 'j','tr_isempty_encoded'), axis=1)
        new_df['j_traingroup'] = new_df.apply(lambda row: get_tr_traingroup_empty_status(row, 'j','tr_traingroup'), axis=1)
    
    return new_df

pri_delay_df= get_empty_freight_trains(pri_delay_df)

sec_delay_df= get_empty_freight_trains(sec_delay_df)
sec_delay_df.head(2)

,i,j,t,c,delayjustifiedbyreason,r0_5min_detection,r1_5min_detection,r2_5min_detection,i_empty,i_traingroup,j_empty,j_traingroup
0,-85230,-86432,3,1137,133,3,6,4,Not Empty,PASSENGER,None,None
1,-69328,-29553,8,1137,282,8,6,4,Empty,PASSENGER,Empty,PASSENGER


In [48]:
pri_delay_df

,i,t,r0_5min_detection,r1_5min_detection,r2_5min_detection,delayjustifiedbyreason,i_empty,i_traingroup
0,-99992,978,114,2,4,9048,Empty,PASSENGER
1,-99992,13085,125,2,5,12423,Empty,PASSENGER
2,-99991,2176,160,6,4,651,Not Empty,FREIGHT
3,-99991,2178,162,6,4,201,Not Empty,FREIGHT
4,-99991,15954,114,5,5,127,Not Empty,FREIGHT
...,...,...,...,...,...,...,...,...
226786,-17,22950,198,1,6,158,Not Empty,PASSENGER
226787,-17,23244,204,2,6,61,Not Empty,PASSENGER
226788,-17,24966,198,1,6,92,Not Empty,PASSENGER
226789,-17,25255,199,2,6,129,Not Empty,PASSENGER


#### Sec delay summary

In [49]:
################################################################################
# The following function extracts the summary statistics of 
## freight and passenger trains in 'i' and 'j' columns
## empty and nonempty trains in the 'i' and 'j' columns
## writes the summary to a local csv file in  working directory
################################################################################

def get_statistics_for_sec_delay_df(new_df):
    
    # 1. Train Group Combinations
    train_group_counts = new_df.groupby(['i_traingroup', 'j_traingroup']).size().reset_index(name='count')
    train_group_counts['Category'] = 'Train Group Combinations'
    train_group_counts['i_empty'] = pd.NA
    train_group_counts['j_empty'] = pd.NA

    # 2. Empty Status Combinations
    empty_status_counts = new_df.groupby(['i_empty', 'j_empty']).size().reset_index(name='count')
    empty_status_counts['Category'] = 'Empty Status Combinations'
    empty_status_counts['i_traingroup'] = pd.NA
    empty_status_counts['j_traingroup'] = pd.NA

    # 3. Individual Value Counts for `i_traingroup`, `j_traingroup`, `i_empty`, `j_empty`
    i_traingroup_counts = new_df['i_traingroup'].value_counts().reset_index(name='count')
    i_traingroup_counts.columns = ['i_traingroup', 'count']
    i_traingroup_counts['Category'] = 'i_traingroup_Counts'
    i_traingroup_counts['j_traingroup'] = pd.NA
    i_traingroup_counts['i_empty'] = pd.NA
    i_traingroup_counts['j_empty'] = pd.NA

    j_traingroup_counts = new_df['j_traingroup'].value_counts().reset_index(name='count')
    j_traingroup_counts.columns = ['j_traingroup', 'count']
    j_traingroup_counts['Category'] = 'j_traingroup_Counts'
    j_traingroup_counts['i_traingroup'] = pd.NA
    j_traingroup_counts['i_empty'] = pd.NA
    j_traingroup_counts['j_empty'] = pd.NA

    i_empty_counts = new_df['i_empty'].value_counts().reset_index(name='count')
    i_empty_counts.columns = ['i_empty', 'count']
    i_empty_counts['Category'] = 'i_empty_Counts'
    i_empty_counts['i_traingroup'] =pd.NA
    i_empty_counts['j_traingroup'] = pd.NA
    i_empty_counts['j_empty'] = pd.NA

    j_empty_counts = new_df['j_empty'].value_counts().reset_index(name='count')
    j_empty_counts.columns = ['j_empty', 'count']
    j_empty_counts['Category'] = 'j_empty_Counts'
    j_empty_counts['i_traingroup'] = pd.NA
    j_empty_counts['j_traingroup'] = pd.NA
    
    j_empty_counts['i_empty'] = pd.NA

    # 4. Combine all into a single DataFrame
    combined_df = pd.concat([
        train_group_counts,
        empty_status_counts,
        i_traingroup_counts,
        j_traingroup_counts,
        i_empty_counts,
        j_empty_counts
    ], ignore_index=True)

    # Reorder columns to match the desired output
    combined_df = combined_df[['Category', 'i_traingroup', 'j_traingroup', 'i_empty', 'j_empty', 'count']]
    
    file_name = os.path.join(DATA_DIR, 'sec_delay_train_statistics_' + TIME_FREQUENCY + '.csv')
    combined_df.to_csv(file_name, index=False)
    
    return combined_df


combined_df = get_statistics_for_sec_delay_df(sec_delay_df)
combined_df

,Category,i_traingroup,j_traingroup,i_empty,j_empty,count
0,Train Group Combinations,FREIGHT,FREIGHT,NaN,NaN,2190
1,Train Group Combinations,FREIGHT,PASSENGER,NaN,NaN,6050
2,Train Group Combinations,PASSENGER,FREIGHT,NaN,NaN,2378
3,Train Group Combinations,PASSENGER,PASSENGER,NaN,NaN,24859
4,Empty Status Combinations,NaN,NaN,Empty,Empty,918
5,Empty Status Combinations,NaN,NaN,Empty,Not Empty,4704
6,Empty Status Combinations,NaN,NaN,Not Empty,Empty,2252
7,Empty Status Combinations,NaN,NaN,Not Empty,Not Empty,27603
8,i_traingroup_Counts,PASSENGER,NaN,NaN,NaN,32210
9,i_traingroup_Counts,FREIGHT,NaN,NaN,NaN,8975


#### Data cleaning for primary/secondary delay

In [50]:
## We perform data cleaning over our pri_delay_df
## We drop all frieght trains and empty trains 
# and store cleaned pri_delay_df to the parquet file 
# in the working directory PRIMARY_DELAY_CLEANED_FILE_PATH , 
# we overwrite our pri_delay_df with the cleaned version that only passenger and non empty trains

def clean_and_store_pri_delay(pri_delay_df):
    print("Pri_df shape before dropping Empty and frieght trains")
    print(pri_delay_df.shape)
    pri_delay_df = pri_delay_df[(pri_delay_df['i_traingroup'] == 'PASSENGER') 
                               & (pri_delay_df['i_empty'] == 'Not Empty')]

    # Columns to convert
    pri_delay_df['delayjustifiedbyreason'] = pri_delay_df['delayjustifiedbyreason'].fillna(0).astype(int)
            
    grouped = pri_delay_df.groupby(['i', 't', 'r0_5min_detection', 'r1_5min_detection', 'r2_5min_detection'])['delayjustifiedbyreason'].max().reset_index()
    
    grouped.sort_values(by='t', inplace=True)
    grouped.reset_index(drop=True, inplace=True)

    grouped.to_parquet(PRIMARY_DELAY_CLEANED_FILE_PATH, index=False)
    print(grouped.shape)

    
clean_and_store_pri_delay(pri_delay_df)

Pri_df shape before dropping Empty and frieght trains
(226791, 8)
(150588, 6)


In [51]:
## We perform data cleaning over our sec_delay_df
## We drop all frieght trains and empty trains 
# and store cleaned sec_delay_df to the parquet file 
# in the working directory SECONDARY_DELAY_CLEANED_FILE_PATH , 
# we overwrite our sec_delay_df with the cleaned version that only passenger and non empty trains

def clean_and_store_sec_delay(sec_delay_df):
    print("Sec_df shape before dropping Empty and frieght trains")
    print(sec_delay_df.shape)
    sec_delay_df = sec_delay_df[(sec_delay_df['i_traingroup'] == 'PASSENGER') 
                               & (sec_delay_df['i_empty'] == 'Not Empty') 
                               & (sec_delay_df['j_traingroup'] == 'PASSENGER') 
                               & (sec_delay_df['j_empty'] == 'Not Empty')]

    # Columns to convert
    sec_delay_df['delayjustifiedbyreason'] = sec_delay_df['delayjustifiedbyreason'].fillna(0).astype(int)
    
    grouped = sec_delay_df.groupby(['i', 'j','t', 'c', 'r0_5min_detection', 'r1_5min_detection', 'r2_5min_detection'])['delayjustifiedbyreason'].max().reset_index()
    
    grouped.sort_values(by='t', inplace=True)
    grouped.reset_index(drop=True, inplace=True)

    grouped.to_parquet(SECONDARY_DELAY_CLEANED_FILE_PATH, index=False)
    print(grouped.shape)

    
clean_and_store_sec_delay(sec_delay_df)


Sec_df shape before dropping Empty and frieght trains
(41229, 12)
(19034, 8)


### Load the cleaning primary/secondary delay

In [9]:
################################################################################
# Call the read_function_data function to read parquet file 
# in the working directory PRIMARY_DELAY_FILE_PATH,SECONDARY_DELAY_CLEANED_FILE_PATH 
# we have created earlier via the function prepare_primary_delay_df, clean_and_store_sec_delay
################################################################################

pri_delay_df = read_function_data(PRIMARY_DELAY_CLEANED_FILE_PATH)
# sec_delay_df = read_function_data(SECONDARY_DELAY_CLEANED_FILE_PATH)


In [5]:
sec_delay_df = read_function_data(SECONDARY_DELAY_CLEANED_FILE_PATH)
sec_delay_df

new_df = sec_delay_df[['i', 'j','t']].copy()
    
print(new_df.shape)
new_df.drop_duplicates(inplace=True)
print(new_df.shape)

(19034, 3)
(18960, 3)


## Construction of the delay varialbe $w_{i,t}$

We define as $y_{i,t}$ the primary delay of train $i$ at time $t$. This is not taken as an endogenous variable of the model, but as a known quantity at time $t-\ell$. In fact, our goal is to be able to predict the cascade of delays caused by primary delays, leading to the total delay of train $i$ at time $t$. We also define as $D_{i}$ as the departure time of tray $i$ and let $R_{i}(t) = \{h \in \mathbb{N} ~:~ t \leq h \leq D_{i} \}$. The total delay of train $i$ at time $t$ is obtained as follows:
$$
w_{i,t} = \begin{cases}
    \underbrace{\sum_{h \in R_{i}(t)} y_{i,h}}_{\text{primary delay}} + \underbrace{\sum_{c \in \mathcal{C}} \sum_{h \in R_{i}(t)} \sum_{j \in \mathcal{E}_{i,t}} w_{i,j,h}^c}_{\text{secondary delay}} - \underbrace{\sum_{h \in R_{i}(t)} b_{i,h}}_{\text{recovery}}, & \text{ if } t \geq D_i\\[1.2cm]
    0 & \text{ otherwise }
\end{cases}
$$

where $\mathcal{E}_{i,t}$ is the list of trains that are neighbours of train $i$ at time $t$. This notion of neighborhood is defined based on the idea of possible infrastructure and resource sharing by each pair of trains. To this end, let $r_0(t)$, $r_1(t)$, and $r_2(t)$ be the respective time of the day (in minutes), the day of the week (from one to seven) and month of the year (from one to seven), corresponding to data time $t$.

### Variable $y_{i,t}$: Primary delay component of $w_{i,t}$

In [55]:
################################################################################
## The following function extracts the primary delay in seconds from pri_delay_df
## for the train 'i' at time 't'
################################################################################

def get_y(i, t, delay_df):
    """
    Check primary delay for a given train 'i' at the time 't'.

    Parameters:
    - i (int): Train identifier.
    - t : Time of interest 
    - delay_df (pd.DataFrame): DataFrame containing delay information with columns 'i', 't', and 'delayjustifiedbyreason'.

    Returns:
    - int: delay in seconds if the delayjustifiedbyreason exists, otherwise returns 0.
    """
    # Filter the DataFrame based on the given train identifier and time
    i_match = delay_df['i'].values == i
    t_match = delay_df['t'].values == t
    
    # Combine the conditions using element-wise 'AND' operation & Find indices where the combined condition is True
    combined_match = i_match & t_match
    matched_indices = np.where(combined_match)[0]
    
    # Check if the filtered DataFrame is not empty
    if len(matched_indices) > 0:
        # Retrieve the delayjustifiedbyreason and return 1 if it's not zero, otherwise return 0
        delay_value = delay_df['delayjustifiedbyreason'].values[matched_indices[0]]
        return delay_value if delay_value != 0 else 0
    else:
        # Handle case where no matching rows are found
        return 0


print(get_y(pri_delay_df.iloc[3]['i'],pri_delay_df.iloc[3]['t'],pri_delay_df))

247


### Variable $y_{i,t}$: calculate the following term:
$$
\underbrace{\sum_{h \in R_{i}(t)} y_{i,h}}_{\text{primary delay}}
$$

In [56]:
################################################################################
# The following function calculate the total primary delay term (the summation above)
################################################################################

def get_primary_delay(i, t, D_i, delay_df=pri_delay_df):
    """
    Calculate the sum of primary delays (y_values) for a given train 'i' over the time range [t, D_i].

    Parameters:
    - i (int): Train identifier.
    - t : Lower bound of the time range.
    - D_i : Train Departure time.
    - delay_df (pd.DataFrame):  DataFrame containing information about train primary delays.

    Returns:
    - int: The sum of primary delays for train 'i' from time 't' to 'D_i'.
    """
    
    result = 0
    for h in range(D_i,t):
        y_value = get_y(i, h, delay_df)

        # Add the y_value to the result
        result += y_value

    return result

# Example usage:
# Calculate the calculate_primary_delay
i = sec_delay_df.iloc[3]['i']
t = sec_delay_df.iloc[3]['t']
print("i",i)
print("t",t)
D_i = int(get_D_it(i, t))
print("D_i",D_i)
get_primary_delay(i, t, D_i)

i -15617
t 75
D_i 74


np.int64(64)

### Variable $w_{i,j,t}^c$: Secondary delay component of $w_{i,t}$:

In [6]:
################################################################################
# The following function returns delay in seconds train j has caused a 
# secondary delay of type c to train i at time t
################################################################################

def get_w_ijtc(i,j,t,c,temp_df=sec_delay_df):
    
    """
    Check secondary delay for a given train 'i' by train 'j' at the time t  due to reason 'c'.

    Parameters:
    - i (int): Train identifier.
    - j : delay causing trains 'j' for a given train 'i'.
    - t : time.
    - c : delay reason c
     Returns:
    - int: delay in seconds if the delayjustifiedbyreason exists, otherwise returns 0.
    """
    
    i_match = temp_df['i'].values == i
    j_match = temp_df['j'].values == j
    t_match = temp_df['t'].values == t
    c_match = temp_df['c'].values == c
    
    # Combine the conditions using element-wise 'AND' operation & Find indices where the combined condition is True
    combined_match = i_match & j_match & t_match & c_match
    matched_indices = np.where(combined_match)[0]
    
    # Check if the filtered DataFrame is not empty
    if len(matched_indices) > 0:
        delay_value = temp_df['delayjustifiedbyreason'].values[matched_indices[0]]
        return delay_value if delay_value != 0 else 0
    else:
        # Handle case where no matching rows are found
        return 0

print(get_w_ijtc(sec_delay_df.iloc[3]['i'],
                 sec_delay_df.iloc[3]['j'],
                 sec_delay_df.iloc[3]['t'],
                 sec_delay_df.iloc[3]['c'],
                 sec_delay_df))

185


#### Filling the EndogenousData class

In [7]:
################################################################################
# The following fuction uses cleaned sec_delay_df to provide values to the columns i, j, c, 't', in the class EndogenousData
# and uses function_w(i,j,t,c,temp_df=sec_delay_df) to provide values to the column w_{i,j,t}^c in the class EndogenousData
################################################################################

def fill_endogenousData_class():
    # Create a new DataFrame with the necessary columns
    new_df = sec_delay_df[['i', 'j', 't', 'c']].copy()
    
    print(new_df.shape)
    new_df.drop_duplicates(inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    
    print(new_df.shape)
    # Apply function_w to each row in the new DataFrame
    new_df['w_{i,j,t}^c'] = new_df.apply(lambda row: get_w_ijtc(row['i'], row['j'], row['t'], row['c']), axis=1)

    # Initialize the EndogenousData class with the new DataFrame
    return EndogenousData(new_df)


# Example of how to use the class
endogenous_data = fill_endogenousData_class()
temp = endogenous_data.get_data()

(19034, 4)
(19034, 4)


#### To construct the secondary delay term of $w_{i,t}$ we need to define the set $\mathcal{C}$, as appearing in the following summation:
$$
\underbrace{\sum_{c \in \mathcal{C}} \sum_{h \in R_{i}(t)} \sum_{j \in \mathcal{E}_{i,t}} w_{i,j,h}^c}_{\text{secondary delay}}
$$


In [59]:
################################################################################
# The following function returns the set of delay reasons C
################################################################################

def get_C_it(i, w_ijtc):
    """
    Get all delay reason 'c' for a given train 'i' from the DataFrame.

    Parameters:
    - i (int): Train identifier.
    - w_ijtc (pd.DataFrame): DataFrame containing information about train secondary delays.

    Returns:
    - set: Set of delay reason 'c' for the given train 'i' at time 't'.
    """
    # Step 1: Check if 'i' & c column matches the given value using NumPy array
    i_match = w_ijtc['i'].values == i
    c_not_null = ~w_ijtc['c'].isnull().values  # Exclude rows where 'c' is NaN

    # Step 2: Combine the two conditions using element-wise 'AND' operation
    combined_match = i_match & c_not_null

    # Step 3: extract 'c' values from the DataFrame
    delay_array = w_ijtc['c'].values[combined_match]    
    return np.unique(delay_array)


#### To construct the secondary delay term of $w_{i,t}$ we need to define the set $\mathcal{E}_{i,t}$, as appearing in the following summation:
$$
\underbrace{\sum_{c \in \mathcal{C}} \sum_{h \in R_{i}(t)} \sum_{j \in \mathcal{E}_{i,t}} w_{i,j,h}^c}_{\text{secondary delay}}
$$

In [60]:
################################################################################
# The following functions calculates the global neighborhoods (network)
#def get_E_i_t(i,t,w_ijtc): # All the train j that where producing a secondary delay to i at time 't'
#def get_E_i_r0(i, r0(t), w_ijtc): # All the train j that where producing a secondary delay to i at all privious days at time r0(t)
#def get_E_i_r1(i, r1(t), w_ijtc): # All the train j that where producing a secondary delay to i at day r1(t)
#def get_E_i_r2(i, r2(t), w_ijtc): # All the train j that where producing a secondary delay to i in month r2(t)
################################################################################

################################################################################
# The following function returns All the train j that where producing a secondary delay to i at all previous minute at r0(t)
################################################################################

def get_E_i_r0(i, r0, w_ijtc):

    """
    Get all causing secondary delay causing trains 'j' for a given train 'i' at all previous days at minute r0(t)

    Parameters:
    - i (int): Train identifier.
    - r0 (timestamp): minute(t).
    - w_ijtc (pd.DataFrame): DataFrame containing information about train secondary delays.

    Returns:
    - set: Set of delay causing trains 'j' for the given train 'i' at minute 'r0'.
    """
    # Step 1: Check if 'i' , 'r0'  column matches the given value using NumPy array
    i_match = w_ijtc['i'].values == i
    r0_match = w_ijtc['r0_5min_detection'].values == r0
    j_not_null = ~w_ijtc['j'].isnull().values  # Exclude rows where 'j' is NaN

    # Step 2: Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & r0_match & j_not_null

    # Step 3: extract 'j' values from the DataFrame
    causing_trains_list = w_ijtc['j'].values[combined_match].tolist()

    # Convert the list to a set to remove duplicate values (if any)
    causing_trains_set = set(causing_trains_list)

    # Return the set of causing trains for the given train 'i'
    return list(causing_trains_set)


################################################################################
# The following function returns All the train j that where producing a secondary delay to i at day r1(t)
################################################################################
def get_E_i_r1(i, r1, w_ijtc):

    """
    Get all secondary delay causing trains 'j' for a given train 'i' at all previous days at day r1

    Parameters:
    - i (int): Train identifier.
    - r1 (timestamp): day(t)
    - w_ijtc (pd.DataFrame): DataFrame containing information about train secondary delays.

    Returns:
    - set: Set of delay causing trains 'j' for the given train 'i' at 'r1'.
    """

    # Step 1: Check if 'i' , 'r1'  column matches the given value using NumPy array
    i_match = w_ijtc['i'].values == i
    r1_match = w_ijtc['r1_5min_detection'].values == r1
    j_not_null = ~w_ijtc['j'].isnull().values  # Exclude rows where 'j' is NaN

    # Step 2: Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & r1_match & j_not_null

    # Step 3: extract 'j' values from the DataFrame
    causing_trains_list = w_ijtc['j'].values[combined_match].tolist()

    # Convert the list to a set to remove duplicate values (if any)
    causing_trains_set = set(causing_trains_list)

    # Return the set of causing trains for the given train 'i'
    return list(causing_trains_set)



################################################################################
# The following function returns All the train j that where producing a secondary delay to i in month r2(t)
################################################################################
def get_E_i_r2(i, r2, w_ijtc):

    """
    Get all secondary delay causing trains 'j' for a given train 'i' in month r2(t)

    Parameters:
    - i (int): Train identifier.
    - r2 (timestamp): month(t)
    - w_ijtc (pd.DataFrame): DataFrame containing information about train secondary delays.

    Returns:
    - set: Set of delay causing trains 'j' for the given train 'i' at 'r2'.
    """

    # Step 1: Check if 'i' , 'r2'  column matches the given value using NumPy array
    i_match = w_ijtc['i'].values == i
    r2_match = w_ijtc['r2_5min_detection'].values == r2
    j_not_null = ~w_ijtc['j'].isnull().values  # Exclude rows where 'j' is NaN

    # Step 2: Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & r2_match & j_not_null

    # Step 3: extract 'j' values from the DataFrame
    causing_trains_list = w_ijtc['j'].values[combined_match].tolist()

    # Convert the list to a set to remove duplicate values (if any)
    causing_trains_set = set(causing_trains_list)

    # Return the set of causing trains for the given train 'i'
    return list(causing_trains_set)

################################################################################
# The following function returns the set E_{i}
################################################################################

def get_E_i(i,  w_ijtc):

    """
    Get all secondary delay causing trains 'j' for a given train 'i'  from the DataFrame.

    Parameters:
    - i (int): Train identifier.
    - w_ijtc (pd.DataFrame): DataFrame containing information about train secondary delays.

    Returns:
    - set: Set of delay causing trains 'j' for the given train 'i'.
    """
    # Step 1: Check if 'i'  column matches the given value using NumPy array
    i_match = w_ijtc['i'].values == i
    j_not_null = ~w_ijtc['j'].isnull().values  # Exclude rows where 'j' is NaN

    # Step 2: Combine the conditions using element-wise 'AND' operation
    combined_match = i_match & j_not_null

    # Step 3: extract 'j' values from the DataFrame
    causing_trains_array = w_ijtc['j'].values[combined_match]
    
    return np.unique(causing_trains_array)


causing_train_list_t = get_E_i(sec_delay_df.iloc[0]['i'], sec_delay_df)
print(causing_train_list_t)

causing_train_list_r2 = get_E_i_r2(sec_delay_df.iloc[0]['i'],sec_delay_df.iloc[0]['r2_5min_detection'], sec_delay_df)
print(causing_train_list_r2)

causing_train_list_r1 = get_E_i_r1(sec_delay_df.iloc[0]['i'],sec_delay_df.iloc[0]['r1_5min_detection'], sec_delay_df)
print(causing_train_list_r1)

causing_train_list_r0 = get_E_i_r0(sec_delay_df.iloc[0]['i'],sec_delay_df.iloc[1]['r0_5min_detection'], sec_delay_df)
print(causing_train_list_r0)

[-92102 -65352 -15782 -13745  -2373]
[-13745, -15782, -65352, -92102, -2373]
[-13745]
[-13745]


#### Variable $w_{i,j,h}^c$: Calculate secondary delay term of the total delay $w_{i,t}$:
$$
\underbrace{\sum_{c \in \mathcal{C}} \sum_{h \in R_{i}(t)} \sum_{j \in \mathcal{E}_{i,t}} w_{i,j,h}^c}_{\text{secondary delay}}
$$

In [61]:
################################################################################
# The following function calculate the secondary delay term (the summation above)
################################################################################

def get_secondary_delay_slow(i, t, D_i, endogenous_data):
    
    """
    - i: Train identifier.
    - t: time
    - D_i: Value for 'D_i' (Departure time of the train)

    Returns:
    - int: The sum of secondary delays for train 'i' from time 't' to 'D_i'.
    """
    if(D_i<0):
        return 0
    else:
        w_ijtc =  endogenous_data.filter_data(i=i)
        E_i = get_E_i(i, w_ijtc)
        #print("All causing trains:: ",E_i)

        C = get_C_it(i, w_ijtc)
        #print("All delay Reason:: ",C)

        result = 0

        # Iterate over sets C, R_i, and E_i
        for c in C:
            for h in range(D_i, t):
                for j in E_i:
                    # Retrieve the value of wc for the given c, i, j, and h
                    # wc_result = function_w(i, j, h, c, sec_delay_df) # Please, replace with the EndogenousData

                    # Filter the data for the given i, j, t, and c from the EndogenousData
                    filtered_data = endogenous_data.filter_data(i=i, j=j, t=h, c=c)

                    # Check if the filtered data is not empty and extract the value of w_{i,j,t}^c
                    if not filtered_data.empty:
                        wc_result = filtered_data['w_{i,j,t}^c'].iloc[0]
                    else:
                        wc_result = 0

                    # Add the wc value to the result
                    result += wc_result

        return result

# Assuming function_w, get_all_causing_trains_j, get_all_delay_type_c are defined
# Calculate the calculate_secondary_delay
i = sec_delay_df.iloc[3]['i']
t = sec_delay_df.iloc[3]['t']

D_i = int(get_D_it(i, t))
print(t, D_i)

result_triple_sum = get_secondary_delay_slow(i, t,D_i, endogenous_data)
print(result_triple_sum)

75 74
0


In [62]:
################################################################################
# The following function calculate the secondary delay term (the summation above)
################################################################################
def get_secondary_delay(i, t, D_i, endogenous_data):
      
    """
    - i: Train identifier.
    - t: time
    - D_i: Value for 'D_i' (Departure time of the train)

    Returns:
    - int: The sum of secondary delays for train 'i' from time 't' to 'D_i'.
    """

    data = endogenous_data.get_data()

    # Filter the data based on the conditions for i, t, and D_i
    # Select rows where 'i' matches input 'i', column 't' having value more than or equal to D_i and less than input 't'
    w_ijtc = data[(data['i'] == i) & 
                  (data['t'] >= D_i) &
                  (data['t'] < t)]

    # Extract list of trains that are neighbours of train i  
    E_i = get_E_i(i, w_ijtc)

    # Extract list of reason that has caused delay to train i  
    C = get_C_it(i, w_ijtc)

    # Create a boolean mask to filter rows where 'j' is in E_i and 'c' is in C
    mask = np.isin(w_ijtc['j'], E_i) & np.isin(w_ijtc['c'], C)

    # Sum the 'w_{i,j,t}^c' values from the DataFrame based on the mask
    return w_ijtc.loc[mask, 'w_{i,j,t}^c'].sum()


# Calculate the calculate_secondary_delay
i = sec_delay_df.iloc[2]['i']
t = sec_delay_df.iloc[2]['t']

D_i = get_D_it(i, t)
print(t, D_i)
result_triple_sum = get_secondary_delay(i, t,D_i, endogenous_data)
print(result_triple_sum)

74 74
0


### Extract Total delay

In [79]:
def create_formatted_data_w_it_with_Dit(delay_df):
    """
    Construct w_it_df with time index t (0..max_t) and train columns i.
    Missing delay values are filled forward only within active times (a_it_df == 1).
    """
    # Build full t range
    max_t = delay_df['t'].max()
    t = np.arange(0, max_t + 1)

    # Unique train IDs
    unique_i = delay_df['i'].unique()

    # Initialize zero DataFrame (rows: t, columns: train IDs)
    w_it_df = pd.DataFrame(0, index=t, columns=unique_i, dtype=int)

    # Aggregate delays
    delay_agg = delay_df.groupby(['t', 'i'])['w(i,t)'].mean().unstack(fill_value=0)

    # Optionally round to nearest integer if you prefer cleaner values
    delay_agg = delay_agg.round().astype(int)
    
    # Reindex to full t range and all trains
    delay_agg = delay_agg.reindex(index=t, columns=unique_i, fill_value=0)

    count = 0
    # Fill within active intervals
    for i in unique_i:
        
        if count % 100 == 0:
            print(count, ":", unique_i.shape[0])
        count += 1
        
        last_val = 0
        for tt in t:
            if a_it_df.loc[tt, i] == 1:  # Train is active
                val = delay_agg.loc[tt, i]
                if val == 0 and last_val != 0:
                    # Fill with previous delay only within active period
                    w_it_df.loc[tt, i] = last_val
                else:
                    w_it_df.loc[tt, i] = val
                    if val != 0:
                        last_val = val
            else:
                # Train inactive — reset delay memory
                last_val = 0
                w_it_df.loc[tt, i] = 0
                
    w_it_df.to_parquet(TOTAL_DELAY_FILE_PATH, index=False)
    return w_it_df

## Read the S_function file we have created earlier
s_r_w_df = read_function_data(FUNCTION_S_R_W0_FILE_PATH)
s_r_w_df

w_it_df = create_formatted_data_w_it_with_Dit(s_r_w_df)
w_it_df.tail(40)
    

0 : 5469
100 : 5469
200 : 5469
300 : 5469
400 : 5469
500 : 5469
600 : 5469
700 : 5469
800 : 5469
900 : 5469
1000 : 5469
1100 : 5469
1200 : 5469
1300 : 5469
1400 : 5469
1500 : 5469
1600 : 5469
1700 : 5469
1800 : 5469
1900 : 5469
2000 : 5469
2100 : 5469
2200 : 5469
2300 : 5469
2400 : 5469
2500 : 5469
2600 : 5469
2700 : 5469
2800 : 5469
2900 : 5469
3000 : 5469
3100 : 5469
3200 : 5469
3300 : 5469
3400 : 5469
3500 : 5469
3600 : 5469
3700 : 5469
3800 : 5469
3900 : 5469
4000 : 5469
4100 : 5469
4200 : 5469
4300 : 5469
4400 : 5469
4500 : 5469
4600 : 5469
4700 : 5469
4800 : 5469
4900 : 5469
5000 : 5469
5100 : 5469
5200 : 5469
5300 : 5469
5400 : 5469


,-85230,-53372,-54740,-20520,-80988,-73561,-37218,-55385,-40482,-65603,...,-60388,-11433,-33079,-33410,-15225,-25821,-74114,-14982,-77146,-15845
26169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
w_it_df.shape

(26209, 5469)

In [85]:
################################################################################
# The following functions `function_w_i_t`, takes   train 𝑖,  time 't' , D_it, w_it_df as input 
# and returns the total delay w(i,t) 
################################################################################

w_it_df = read_function_data(TOTAL_DELAY_FILE_PATH)
w_it_df.columns = w_it_df.columns.astype(int)
def get_wit_observed(i,t,D_it,row=None, delay_df=w_it_df):
    """
    Find total delay for a given train 'i' at the time t.

    Parameters:
    - i (int): Train identifier.
    - t : time.
    - D_i: Value for 'D_i' (Departure time of the train)
    
     Returns:
    - int : total delay at time t .
    """
    
    if row is not None:
        i = row['i']
        t = row['t']
        D_it = row['D_it']
    
    if t >= D_it:
        return delay_df[i].iloc[t-1]
    else:
        return 0


### calculate recovery

In [19]:
################################################################################
# The following function calculates the total recovery by subtracting 
# the primary_delay and secondary_delay from the w_it
################################################################################

def get_recovery(i, t,D_i,Endogenous_data):

    """
    Calculate recovery for a given train 'i' at t .

    Parameters:
    - i (int): Train identifier.
    - t: timestamp
    - D_i: Value for 'D_i' (Departure time of the train)

    Returns:
    - int: total recovery
    """
     
    primary_delay_sum = get_primary_delay(i, t, D_i)
    secondary_delay_sum = get_secondary_delay(i,t,D_i,Endogenous_data)
    w_it = get_wit_observed(i,t,D_i)

    print("w_it observed :: ",w_it)

    recovery = w_it- (primary_delay_sum +secondary_delay_sum)
    return recovery


# Example usage:
i = sec_delay_df.iloc[2]['i']
t = sec_delay_df.iloc[2]['t']
D_i = get_D_it(i, t)
print(i, t, D_i)
get_recovery(i, t,D_i,endogenous_data)

-76539 74 74


NameError: name 'endogenous_data' is not defined

In [76]:
################################################################################
# The following function calculates the total delay by adding 
# the primary_delay, secondary_delay and recovery
################################################################################

def get_wit_estimated(i, t, D_i, Endogenous_data):
    
    """
    Calculate recovery for a given train 'i' at t .

    Parameters:
    - i (int): Train identifier.
    - t: timestamp
    - D_i: Value for 'D_i' (Departure time of the train)

    Returns:
    - int: total recovery
    """
    
    primary_delay_sum = get_primary_delay(i, t, D_i)
    secondary_delay_sum = get_secondary_delay(i,t,D_i, Endogenous_data)
    recovery = get_recovery(i, t, D_i,Endogenous_data)
    
    w_it = primary_delay_sum + secondary_delay_sum + recovery
    
    #print("w_it estimated :: ",w_it)
    return w_it


# Example usage:
i = sec_delay_df.iloc[1]['i']
t = sec_delay_df.iloc[1]['t']

D_i = get_D_it(i, t)
print(t, D_i)
get_wit_estimated(i,t,D_i, endogenous_data)

73 73
w_it observed ::  238


np.int64(238)

## Compute x_tilde

$$
x(j,t) = \tilde{x}(w_{j,t}(W, Y, R), \theta_{t,s}) = \left\{\begin{array}{lll}
     1 & \displaystyle   \quad \text{if } w_{j,t}(W, Y, R) \geq \theta_{\boldsymbol{r}(t)}   \\[0.5cm]
     0 & \quad \text{otherwise } 
\end{array} \right.
$$

In [77]:
################################################################################
# The following function is to fill the x_tilde for our regression input
################################################################################

def get_x_tilde(w_it, theta):
    
    """
    Calculate get_x_tilde for a given 'w_it' and theta.

    Parameters:
    - w_it (int): Total delay of the train.
    - theta: threshold

    Returns:
    - boolean : 
    """

    return 1 if w_it > theta else 0



## Fill regression input for Endogenous data with all 't'

### Fill G for regression all 't'

In [79]:
################################################################################
# The following uses the endogenous data and computes 
# 'gplus_0', 'gplus_1', 'gplus_2', 'gminus_0', 'gminus_1', 'gminus_2' get_S_t_w0_optimized_helper
# store the result retrived for every input by keeping the corresponding combination of input 
# (i, time_i, m)   (j, time_j, m) as key 
# and output of get_S_t_w0 as value in delay_dictionary
################################################################################ 
pd.set_option('mode.chained_assignment', None)

# Get the DataFrame with 'i' and 'j' columns
df = endogenous_data.get_data()[['i', 'j']].drop_duplicates()

# Convert to numpy array and sort by 'i'
unique_pairs = df.to_numpy()
unique_pairs = unique_pairs[unique_pairs[:, 0].argsort()]

t_values = range(0, a_it_df.index.max())

tw_mapping = {}

for i, g in s_r_w_df.groupby("i"):
    # drop duplicates just in case
    tw_mapping[i] = dict(zip(g["t"].values, g["t-w(i,t)"].values))

            
def generate_G_data(range_number, per_job):
    ## setting the tau value
    τ = 3
    directory_path = r'C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data\G_tau33' 
    os.makedirs(directory_path, exist_ok=True)

    from datetime import datetime
    columns = ['gplus_0', 'gplus_1', 'gplus_2', 'gminus_0', 'gminus_1', 'gminus_2']

    # file_list = [
    #     'G_values_batch_50_0_all_t.parquet',
    #     'G_values_batch_50_3000_all_t.parquet',
    #     'G_values_batch_50_6000_all_t.parquet',
    #     'G_values_batch_50_9000_all_t.parquet',
    #     'G_values_batch_50_12000_all_t.parquet'
    # ]

    batch_size = 50
    file_name = 'G_values_batch_50_' + str(range_number) + '_all_t.parquet'
    
    # Process in batches of unique (i, j) pairs
    for start in range(range_number, min(range_number + per_job, len(unique_pairs)), batch_size):# len(unique_pairs), batch_size):
        print("Batch number ::",start)

        
        # # Assign file name
        # if 0 <= start <= 2999:
        #     file_name = 'G_values_batch_50_0_all_t.parquet'
        # elif 3000 <= start <= 5999:
        #     file_name = 'G_values_batch_50_3000_all_t.parquet'
        # elif 6000 <= start <= 8999:
        #     file_name = 'G_values_batch_50_6000_all_t.parquet'
        # elif 9000 <= start <= 11999:
        #     file_name = 'G_values_batch_50_9000_all_t.parquet'
        # else:
        #     file_name = 'G_values_batch_50_12000_all_t.parquet'

        # print(file_name)

        file_path = os.path.join(directory_path, file_name) 
        print("Start time:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        # Initialize the results list for the current batch
        batch_results = []

        # Select the batch of (i, j) pairs
        batch_pairs = unique_pairs[start:start + batch_size]

        # Extract i and j values
        batch_i = np.unique(batch_pairs[:, 0])
        corresponding_j = np.unique(batch_pairs[:, 1])

        # print("Batch i values:", batch_i)
        # print("j values:", corresponding_j)

        # Get unique i values for this batch
        i_values = np.unique(np.concatenate([batch_i, corresponding_j]))
        # print("Unique i_j values for this batch:", i_values)

        # Get stop and passing results
        print("Going to fill")
        delay_stop_1, delay_passing_1 = get_S_t_w0_optimized_helper(i_values, 'delay')
        schedule_stop_1, schedule_passing_1 = get_S_t_w0_optimized_helper(i_values, 'schedule')

        τ_plus_1 = τ + 1 
        min_t = a_it_df.index.min() - τ_plus_1
        max_t = a_it_df.index.max() + τ_plus_1

        # Initialize delay dictionary
        delay_dictionary = {}

        ## Wei modifies, use loc rather than iloc
        def update_delay_dictionary(train_id):
            for t in range(min_t, max_t):
                if t in delay_stop_1[train_id].index:
                    delay_dictionary[(train_id, t, "stop")] = delay_stop_1[train_id].loc[t]
                    delay_dictionary[(train_id, t, "passing")] = delay_passing_1[train_id].loc[t]
                else:
                    delay_dictionary[(train_id, t, "stop")] = np.zeros(len(S_set_stations), dtype=np.int8)
                    delay_dictionary[(train_id, t, "passing")] = np.zeros(len(S_set_stations), dtype=np.int8)


        schedule_dictionary = {}

        ## Wei modifies, use loc rather than iloc
        def update_schedule_dictionary(train_id):
            for t in range(min_t, max_t):
                if t in schedule_stop_1[train_id].index:
                    schedule_dictionary[(train_id, t, "stop")] = schedule_stop_1[train_id].loc[t]
                    schedule_dictionary[(train_id, t, "passing")] = schedule_passing_1[train_id].loc[t]
                else:
                    schedule_dictionary[(train_id, t, "stop")] = np.zeros(len(S_set_stations), dtype=np.int8)
                    schedule_dictionary[(train_id, t, "passing")] = np.zeros(len(S_set_stations), dtype=np.int8)

        
        # Calculate the total number of combinations
        num_combinations = len(batch_pairs) * len(t_values)

        # Preallocate DataFrame
        temp = pd.DataFrame(index=range(num_combinations), columns=columns + ['i', 'j', 't'])

        # Index counter
        index_counter = 0

        # Iterate over each combination of i and j in the batch
        for i, j in batch_pairs:
            # print(i,j, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            if i != j:
                if (i, 0, "stop") not in delay_dictionary:
                    update_delay_dictionary(i)
                    update_schedule_dictionary(i)
                if (j, 0, "stop") not in delay_dictionary:
                    update_delay_dictionary(j)
                    update_schedule_dictionary(j)

                for t in t_values:
                    # Compute results using get_G_helper_optimized
                    m0_minus, m1_minus, m2_minus = get_G_minus_helper_optimized(i, j, t, delay_dictionary, τ)
                    
                    ti = tw_mapping[i].get(t, t)
                    tj = tw_mapping[j].get(t, t)
                    m0_plus, m1_plus, m2_plus = get_G_plus_helper_optimized(i, j, ti, tj, schedule_dictionary,τ)

                    # Store results directly in preallocated DataFrame
                    temp.iloc[index_counter] = [m0_plus, m1_plus, m2_plus, m0_minus, m1_minus, m2_minus,i, j, t,]
                    index_counter += 1

        # Remove empty rows if any (possible if index_counter < num_combinations)
        temp = temp.head(index_counter)

        #  Append directly to disk — don't read the old file
        if not os.path.exists(file_path):
            temp.to_parquet(file_path, index=False)
        else:
            temp.to_parquet(file_path, index=False, append=True, engine="fastparquet")

        #  Free up memory after each batch
        del temp
        import gc; gc.collect()

        
        # file_exists = os.path.isfile(file_path)
        # if file_exists:
        #     # Load existing data
        #     existing_data = pd.read_parquet(file_path)
        #     # Concatenate with new data
        #     combined_data = pd.concat([existing_data, temp])
        # else:
        #     combined_data = temp

        # # Save the combined data back to the Parquet file
        # combined_data.to_parquet(file_path, index=False)
        print("Batch number completed ::",start)
        
# generate_G_data(1300)

In [80]:
from joblib import Parallel, delayed

per_job = 3000
starts = range(0, len(unique_pairs), per_job)

n_jobs = len(starts)
Parallel(n_jobs, backend="loky")(
    delayed(generate_G_data)(s, per_job) for s in starts
)

[None, None, None, None, None]

In [69]:
################################################################################
# In the above step, we extracted G column values for all i, j combinations.
# In the below step, we extract all valid rows based on the G column values.
# 
# Criteria to consider a row is valid or not:
# - A row is considered valid if any of the G columns 
#   (gplus_0, gplus_1, gplus_2, gminus_0, gminus_1, gminus_2) has a value > 0.
#
# Process:
# - Read the data
# - Extract rows meeting the validity criteria.
# - Combine all valid rows into a single DataFrame.
# - Optionally save the resulting DataFrame to a file to the specified path).
################################################################################

def extract_valid_G_rows(directory_path=None, save_path = None):
    
    file_list = [
        'G_values_batch_50_0_all_t.parquet',
        'G_values_batch_50_3000_all_t.parquet',
        'G_values_batch_50_6000_all_t.parquet',
        'G_values_batch_50_9000_all_t.parquet',
        'G_values_batch_50_12000_all_t.parquet'
    ]
    
    temp_path = "temp_valid.parquet"
    unique_pairs_G = pd.DataFrame()
    
    if os.path.exists(temp_path):
        os.remove(temp_path)

    for file_name in file_list:  
        final_df_valid = pd.DataFrame()
        if directory_path is not None:
            # Construct the full file path
            file_path = os.path.join(directory_path, file_name)        
            print(file_path)

            # Read the Parquet file from the specific directory
            temp = pd.read_parquet(file_path, engine="fastparquet")
        
        else:
            temp = pd.read_parquet(file_name, engine="fastparquet")
            print(file_name)
            
        columns_to_check = ['gplus_0', 'gplus_1', 'gplus_2', 'gminus_0', 'gminus_1', 'gminus_2']

        # Extract unique (i, j) pairs
        pairs = temp[['i', 'j']].drop_duplicates()

        # The following line is to check if any G column > 0, and consider it in neighborhood
        ## Rerun this part
        filtered_df = temp[temp[columns_to_check].gt(0).any(axis=1)]
        
        file_exists = os.path.isfile(temp_path)
        if file_exists:
            # Load existing data
            final_df_valid = pd.read_parquet(temp_path)
            # Concatenate the filtered DataFrame to final_df_valid
            final_df_valid = pd.concat([final_df_valid, filtered_df], ignore_index=True)
        else:
            final_df_valid = filtered_df
           
        final_df_valid = final_df_valid.drop_duplicates()
        final_df_valid.to_parquet(temp_path, index=False)
   
        
    if os.path.exists(temp_path):
        os.remove(temp_path)
   
    if save_path is not None:
        # Save the resulting DataFrame to a Parquet file
        save_path_full = os.path.join(directory_path, save_path) 
        final_df_valid.to_parquet(save_path_full, index=False)
    
    print(final_df_valid.shape)
    #return final_df_valid

In [70]:
################################################################################
# This function verifies how many rows of endogenous_data are present in g_df.
# Specifically, it checks how many endogenous data pairs have valid G combinations 
# by comparing the (i, j, t) values in both DataFrames.
# 
# The output provides:
# - The total count of matches and mismatches.
# - A DataFrame showing which (i, j, t) pairs exist in g_df.
################################################################################
def sanity_check_g_df(g_df):
    print(g_df.shape)

    # First, check if i, j, t from endogenous_data exists in g_df
    matched_df = pd.merge(endogenous_data.get_data()[['i', 'j', 't']], 
                           g_df[['i', 'j', 't']], 
                           on=['i', 'j', 't'], 
                           how='left', 
                           indicator=True)
    
    # Check which rows in endogenous_data.get_data() are found in g_df
    matched_df['exists_in_df1'] = matched_df['_merge'] == 'both'

    print("Count in in endogenous_data.get_data() found in g_df:")
    print(matched_df['exists_in_df1'].value_counts())
    #print(matched_df[['i', 'j', 't', 'exists_in_df1']])

    # For rows where exists_in_df1 is False, check if reverse (j, i, t) exists in g_df
    matched_df['exists_in_reverse_df'] = False  # Initialize the column with False
    
    # Iterate through rows where exists_in_df1 is False
    for index, row in matched_df[matched_df['exists_in_df1'] == False].iterrows():
        # Check if the reversed combination (j, i, t) exists in g_df
        reversed_match = g_df[(g_df['j'] == row['i']) & (g_df['i'] == row['j']) & (g_df['t'] == row['t'])]
        
        if not reversed_match.empty:
            matched_df.at[index, 'exists_in_reverse_df'] = True
    
    print("Rows with reversed match (j, i, t) found in g_df:")
    print(matched_df[['i', 'j', 't', 'exists_in_df1', 'exists_in_reverse_df']])

    # Count occurrences of rows where the match is found in either exists_in_df1 or exists_in_reverse_df
    count_exists_in_either = matched_df[['exists_in_df1', 'exists_in_reverse_df']].sum(axis=1).value_counts()

    print("Count of rows present in either exists_in_df1 or exists_in_reverse_df:")
    print(count_exists_in_either)

    return matched_df, count_exists_in_either

### For tau3

In [81]:
# Extract valid G row
# directory_path = os.path.join(DATA_DIR, 'G_tau' + str(τ)) 
directory_path = os.path.join(DATA_DIR, 'G_tau33') 
# valid_G_save_path="G_values_all_valid_tau2.parquet"
valid_G_save_path='G_values_all_valid_tau' + str(τ) + '.parquet'
extract_valid_G_rows(directory_path,valid_G_save_path)

C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data\G_tau33\G_values_batch_50_0_all_t.parquet
C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data\G_tau33\G_values_batch_50_3000_all_t.parquet
C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data\G_tau33\G_values_batch_50_6000_all_t.parquet
C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data\G_tau33\G_values_batch_50_9000_all_t.parquet
C:\Users\w.bi\OneDrive - IESEG\Desktop\Domino\data\G_tau33\G_values_batch_50_12000_all_t.parquet
(4333809, 9)


In [82]:
################################################################################
## we read the file "G_values_all_valid_tau3.parquet" generated in the above step as g_df 
## We drop all the rows that has either (one inactive train) i inactive or j inactive
###############################################################################
# directory_path = os.path.join(DATA_DIR, 'G_tau' + str(τ)) 
directory_path = os.path.join(DATA_DIR, 'G_tau33') 
valid_G_save_path='G_values_all_valid_tau' + str(τ) + '.parquet'

g_df =  read_function_data(os.path.join(directory_path, valid_G_save_path))
print(g_df.shape)

## We drop all the that has either i inactive or j inactive
g_df['a_it'] = g_df.apply(lambda row: get_a_it('i', 't', row), axis=1)
g_df['a_jt'] = g_df.apply(lambda row: get_a_it('j', 't', row), axis=1)

g_df_filtered = g_df[(g_df['a_it'] != 0) & (g_df['a_jt'] != 0)]
matched_df,count_exists_in_either =  sanity_check_g_df(g_df_filtered)
matched_df.to_parquet('Delayed_train_with_valid_G_tau' + str(τ) + '.parquet', index=False)

(4333809, 9)
(3544743, 11)
Count in in endogenous_data.get_data() found in g_df:
exists_in_df1
True     18777
False      257
Name: count, dtype: int64
Rows with reversed match (j, i, t) found in g_df:
           i       j     t  exists_in_df1  exists_in_reverse_df
0     -43095  -13745    73           True                 False
1     -96837  -73430    73           True                 False
2     -76539  -96837    74           True                 False
3     -15617  -73430    75           True                 False
4     -86247  -57615    75           True                 False
...      ...     ...   ...            ...                   ...
19029 -97988  -84068  8630           True                 False
19030 -97988  -84068  8631           True                 False
19031 -97224  -49394  8634           True                 False
19032 -18530   -3873  8635           True                 False
19033 -86289  -51298  8636           True                 False

[19034 rows x 5 columns]
Count

In [75]:
temp = matched_df[matched_df['exists_in_df1']==0]
temp

,i,j,t,_merge,exists_in_df1,exists_in_reverse_df
66,-6089,-22532,111,left_only,False,False
75,-13869,-39859,116,left_only,False,False
86,-58780,-13252,119,left_only,False,False
179,-51208,-12350,168,left_only,False,False
211,-60939,-50524,197,left_only,False,False
...,...,...,...,...,...,...
18969,-15181,-27755,8596,left_only,False,False
18976,-87423,-85801,8598,left_only,False,False
19000,-82704,-71541,8615,left_only,False,False
19008,-51298,-96659,8618,left_only,False,False


In [83]:
################################################################################
# This function extracts relevant G_column values from g_df for each i, j, c pair 
# in the endogenous_data DataFrame. It combines these values with the endogenous_data 
# and saves the result to a Parquet file at the specified path (ENDODATA_WITH_G_FILE_PATH).
# 
# The resulting file will be used to populate regression input fields such as:
# - primarydelay
# - secondarydelay
# - recovery
# - total_delay
################################################################################

pd.set_option('mode.chained_assignment', None)
def process_row(pair, g_df):
    # Filter the relevant rows in g_df
    df = g_df[(g_df['i'] == pair['i']) & (g_df['j'] == pair['j'])]
    df['c'] = pair['c']

    # Create the filtered_df excluding the current 't'
    filtered_df = df[df['t'] != pair['t']].copy()
    filtered_df['w_{i,j,t}^c'] = 0

    # Current pair data where 't' matches
    curr_df = df[df['t'] == pair['t']].copy()
    curr_df['w_{i,j,t}^c'] = pair['w_{i,j,t}^c']

    return pd.concat([curr_df, filtered_df])

def fill_G_on_Endogenous_data(g_df, endogenous_data,save_path=None):
    # Apply the process_row function to each row of endogenous_data
    print(g_df.shape)
    output_list = endogenous_data.get_data().apply(lambda row: process_row(row, g_df), axis=1)

    # Concatenate all the DataFrames in the output list
    output = pd.concat(output_list.tolist(), ignore_index=True)

    # Sort the final output (optional if needed)
    output = output.sort_values(by=['i', 'j', 't'])
    
    if save_path is not None:
        # Save the result to a parquet file
        output.to_parquet(save_path, index=False)
        
    return output

tmax_sec_delay = sec_delay_df['t'].max()
g_df_filtered = g_df_filtered[(g_df_filtered['a_it'] <= tmax_sec_delay)]
fill_G_on_Endogenous_data(g_df_filtered, endogenous_data, save_path=ENDODATA_WITH_G_FILE_PATH)

(3544743, 11)


,gplus_0,gplus_1,gplus_2,gminus_0,gminus_1,gminus_2,i,j,t,a_it,a_jt,c,"w_{i,j,t}^c"
3930509,1,0,0,0,0,0,-99970,-71996,831,1,1,1134,0
3930510,0,0,0,1,0,0,-99970,-71996,833,1,1,1134,0
3930511,0,0,2,0,0,0,-99970,-71996,1119,1,1,1134,0
3930512,0,1,3,0,0,0,-99970,-71996,1120,1,1,1134,0
3930513,0,0,1,0,0,3,-99970,-71996,1121,1,1,1134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357200,1,0,1,1,1,3,-17,-9056,26120,1,1,1137,0
1357201,0,0,0,0,1,5,-17,-9056,26121,1,1,1137,0
1357202,0,0,0,2,0,1,-17,-9056,26122,1,1,1137,0
1357203,0,0,0,1,0,1,-17,-9056,26123,1,1,1137,0


### Filling delay for Regression

In [86]:
def get_wjt_observed(j,t,D_it,row=None, delay_df=w_it_df):
    """
    Find total delay for a given train 'j' at the time t.

    Parameters:
    - j (int): Train identifier.
    - t : time.
    - D_i: Value for 'D_i' (Departure time of the train)
    
     Returns:
    - int : total delay at time t .
    """
    
    if row is not None:
        j = row['j']
        t = row['t']
        D_it = row['D_it']
    
    if t >= D_it:
        return delay_df[j].iloc[t]
    else:
        return 0

In [89]:
################################################################################
# In this step, we process the data from the file ENDODATA_WITH_G_FILE_PATH 
# (generated in the previous step) and compute additional metrics for each row.
#
# Steps:
# 1. Read the data file ENDODATA_WITH_G_FILE_PATH into a DataFrame (`result`).
# 2. Compute the departure time ('D_it') for each train using the `get_D_it` function.
# 3. Calculate the following delay metrics:
#    - Secondary delay: Using the `get_secondary_delay` function.
#    - Primary delay: Using the `get_primary_delay` function.
# 4. Save the updated DataFrame, including the delay metrics, to the file 
#    ENDODATA_WITH_G_PRI_SEC_DELAY_FILE_PATH for further use.
################################################################################

result  =  read_function_data(ENDODATA_WITH_G_FILE_PATH)    
result['D_it'] = result.apply(lambda row: get_D_it('i', 't', row), axis=1)
result['D_it'] = result['D_it'].astype(int)

# print("Filling secondary delay", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# result['secondary_delay'] = result.apply(lambda row: get_secondary_delay(row['j'], row['t'], row['D_it'],endogenous_data), axis=1)
# # result['secondary_delay'] = result.apply(lambda row: get_secondary_delay(row['i'], row['t'], row['D_it'],endogenous_data), axis=1)

# print("Filling primary delay", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# result['primary_delay'] = result.apply(lambda row: get_primary_delay(row['j'], row['t'], row['D_it']), axis=1)
# # result['primary_delay'] = result.apply(lambda row: get_primary_delay(row['i'], row['t'], row['D_it']), axis=1)
result.to_parquet(ENDODATA_WITH_G_PRI_SEC_DELAY_FILE_PATH, index=False)

In [90]:
################################################################################
# In this step, we process the data containing secondary delay and primary delay
# (from the file ENDODATA_WITH_G_PRI_SEC_DELAY_FILE_PATH) to compute additional metrics.
#
# Steps:
# 1. Read the data file ENDODATA_WITH_G_PRI_SEC_DELAY_FILE_PATH into a DataFrame (`result`).
# 2. Remove duplicate rows based on the columns combination ['i', 'j', 't', 'c'].
# 3. Compute the below metrics:
#    - `wit_observed`: Using the `get_wit_observed` function.
#    - `recovery`: Calculated as the difference between `wit_observed` and the 
#       sum of `primary_delay` and `secondary_delay`.
#    - `wit_estimated`: Calculated as the sum of `primary_delay`, 
#       `secondary_delay`, and `recovery`.
# 4. Save the updated DataFrame to the file 
#    REGRESSION_INPUT_VALID_G_FILE_PATH to use for regression.
################################################################################

result = read_function_data(ENDODATA_WITH_G_PRI_SEC_DELAY_FILE_PATH)

### bug checked
result = result.sort_values("w_{i,j,t}^c", ascending=False)
result = result.drop_duplicates(subset=['i', 'j', 't', 'c'])
result = result.sort_index()

# result = result.drop_duplicates(subset=['i', 'j', 't', 'c'])
result = result.reset_index(drop=True)


## change from wit to wjt
print("Filling wjt_observed", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
result['wjt'] = result.apply(lambda row: get_wjt_observed('j', 't','D_it',row), axis=1)
result['wit-1'] = result.apply(lambda row: get_wit_observed('i', 't','D_it',row), axis=1)


# print("Filling recovery", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# result['recovery'] = result.apply(lambda row: row['wit_observed']-(row['primary_delay'] + row['secondary_delay']), axis=1)
# result['wit_estimated'] = result.apply(lambda row: (row['primary_delay'] + row['secondary_delay']) + row['recovery'], axis=1)

result.to_csv(REGRESSION_INPUT_VALID_G_FILE_PATH, index=False)

Filling wjt_observed 2025-12-21 04:40:16


In [91]:
def compute_wjt_deltas(w_it_df):
    """Compute Δwjt, Δwjt-1, Δwjt-2, Δwjt-3 for each train j."""
    w_it_df = w_it_df.sort_index()
    # delta_0 = wjt - wjt-1
    delta_0 = w_it_df.diff(1)
    
    # delta_1 = wjt-1 - wjt-2
    delta_1 = w_it_df.shift(1).diff(1)
    
    # delta_2 = wjt-2 - wjt-3
    delta_2 = w_it_df.shift(2).diff(1)
    
    # delta_2 = wjt-2 - wjt-3
    delta_3 = w_it_df.shift(3).diff(1) 
    
    delta_0 = delta_0.fillna(0)
    delta_1 = delta_1.fillna(0)
    delta_2 = delta_2.fillna(0)
    delta_3 = delta_3.fillna(0)
    
    return {
        'Δwjt': delta_0,
        'Δwjt-1': delta_1,
        'Δwjt-2': delta_2,
        'Δwjt-3': delta_3
    }

deltas = compute_wjt_deltas(w_it_df)

delta_merge_df = (
    pd.concat({
        name: ddf.stack()
        for name, ddf in deltas.items()
    }, axis=1)
    .reset_index()
)
delta_merge_df.columns = ['t', 'j', 'Δwjt', 'Δwjt-1', 'Δwjt-2', 'Δwjt-3']

In [92]:
gc.collect()

9

In [93]:
# Make sure both are sorted by ['j', 't']
result = result.sort_values(['j', 't'])
delta_merge_df = delta_merge_df.sort_values(['j', 't'])

# Initialize an empty list to collect results
merged_parts = []

# Loop through each j (train) to merge only relevant rows
for j_val, part in result.groupby('j'):
    delta_part = delta_merge_df[delta_merge_df['j'] == j_val]
    merged = pd.merge(part, delta_part, on=['t', 'j'], how='left')
    merged_parts.append(merged)

# Concatenate all small merged results
result = pd.concat(merged_parts, ignore_index=True)

# Fill missing deltas with 0
for col in ['Δwjt', 'Δwjt-1', 'Δwjt-2', 'Δwjt-3']:
    if col in result.columns:
        result[col] = result[col].fillna(0)

In [31]:
# result = result.merge(delta_merge_df, on=['t', 'j'], how='left')
# result[['Δwjt', 'Δwjt-1', 'Δwjt-2', 'Δwjt-3']] = result[['Δwjt', 'Δwjt-1', 'Δwjt-2', 'Δwjt-3']].fillna(0)

In [29]:
result

,gplus_0,gplus_1,gplus_2,gminus_0,gminus_1,gminus_2,i,j,t,a_it,a_jt,c,"w_{i,j,t}^c",D_it,wjt,wit-1,Δwjt,Δwjt-1,Δwjt-2,Δwjt-3
0,1,1,0,0,0,0,-39113,-99970,232,1,1,1134,0,205,12,8,-26.0,10.0,28.0,0.0
1,0,0,0,1,1,0,-39113,-99970,233,1,1,1134,0,205,12,-54,0.0,-26.0,10.0,28.0
2,1,1,1,0,0,0,-39113,-99970,234,1,1,1134,0,205,12,-64,0.0,0.0,-26.0,10.0
3,4,1,0,2,1,0,-92825,-99970,235,1,1,1136,0,235,19,0,7.0,0.0,0.0,-26.0
4,0,0,0,1,1,1,-39113,-99970,235,1,1,1134,0,205,19,-20,7.0,0.0,0.0,-26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409873,1,0,0,0,0,0,-7078,-17,26130,1,1,1137,0,26122,-122,20,0.0,-44.0,-11.0,-63.0
2409874,4,2,2,1,0,0,-7078,-17,26131,1,1,1137,0,26122,18,6,140.0,0.0,-44.0,-11.0
2409875,3,1,2,4,2,2,-7078,-17,26132,1,1,1137,0,26122,22,21,4.0,140.0,0.0,-44.0
2409876,2,1,0,3,1,2,-7078,-17,26133,1,1,1137,0,26122,22,31,0.0,4.0,140.0,0.0


In [94]:
result.to_csv(REGRESSION_INPUT_VALID_G_DALTA_FILE_PATH , index=False)